# Frankross_analytic Github

- [frankross_analytic_github link](https://github.com/kr2rohit/frankross_analytics)

# Backup At : 4th Feb2025

# Orders Data update ,  Updated_at : 24th feb25

In [ ]:
# Version: 1.02
#(Removed Coupon data as they had duplicates due which duplicates orders were recorder in c42 combined and leads to double rporting)
# Updated at 5th March25
# updates: added promotion_id and coupon_code 
# This Script Updates current month order in Summary sheet , CC Dashboar, Re order sheet, - Logging Configured 
import schedule as schedule
import time
import logging

logging.basicConfig(
    filename="log_Order_summary_update.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def Order_summary_update():
    try:
        import csv
        import pandas as pd
        import numpy as np
        import psycopg2
        import pandas as pd
        import numpy as np
        import paramiko
        import re
        from sshtunnel import SSHTunnelForwarder
        import gspread
        from gspread_dataframe import set_with_dataframe
        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                        database='efrprod',
                        user='emamireaduser',
                        password='emamireadaccess'
                )
                cursor = connection.cursor()
        
                logging.info('Executing DB query.....')
                cursor.execute(
                '''
                SELECT order_Table.*
                ,CASE WHEN
                	od IS NULL THEN 'Not First Order'
                	ELSE '1st Order' END AS new_flag
                ,CASE
                 	WHEN timeout_at IS NOT NULL THEN 'timeout'
                	WHEN rejected_at IS NOT NULL THEN 'rejected'
                	WHEN accepted_at IS NOT NULL THEN 'accepted'
                	ELSE 'pending' end as order_action
                --,promo.coupon_code
                FROM
                (SELECT		
                 osl.order_id		
                ,o.user_id as created_by_id		
                ,CASE		
                	WHEN o.state =0 THEN 'cart'	
                	WHEN o.state =1 THEN 'pre_checkout'	
                	WHEN o.state =2 THEN 'Checkout'	
                	WHEN o.state =6 THEN 'Delivered'	
                	WHEN o.state =7 THEN 'Cancelled'	
                	WHEN o.state =14  THEN 'Return_completed'	
                	ELSE 'In-Progress'	
                END AS status		
                ,o.city_id		
                ,o.delivery_remarks		
                ,o.payment_method		
                ,o.auto_completed		
                ,o.order_total_paise*1.0/100 AS order_value		
                ,o.shipping_total_paise*1.0/100 AS Shipping_charge		
                ,o.doctor_names		
                --,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id		
                ,osl.confirmed_on		
                ,CASE		
                	WHEN osl.channel = 0 THEN 'Call Center'	
                	WHEN osl.channel = 1 THEN 'Mobile'	
                	ELSE 'Website'	
                END as channel_name		
                ,CASE		
                	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'	
                	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                	WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                	WHEN osl.channel = 0 THEN 'Call Center'	
                	WHEN osl.channel = 1 THEN 'Saathi App'	
                	ELSE 'Website'	
                END as channel2		
                ,CASE 		
                	WHEN o.state =0 THEN 'cart'	
                	WHEN o.state =1 THEN 'pre_checkout'	
                	WHEN o.state =2 THEN 'Checkout'	
                	WHEN o.state =3 THEN 'order_received'	
                	WHEN o.state =4 THEN 'shipped'	
                	WHEN o.state =5 THEN 'out_for_delivery'	
                	WHEN o.state =6 THEN 'Delivered'	
                	WHEN o.state =7 THEN 'Cancelled'	
                	WHEN o.state =14 THEN 'Return_completed'	
                	WHEN o.state =17 THEN 'rescheduled'	
                		ELSE 'In-Progress' END AS detailed_status
                ,customer_id		
                ,users.name AS modified_by		
                ,w.code AS fulfillment_center	
                ,Wa.code AS Actual_Mapped_Dc
                ,DS.slot_description		
                ,DS.slot_date AS Expected_Delivery		
                ,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start		
                ,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end		
                ,co.state_changed_on + time '5:30' as cancelled_date		
                ,co.reason		
                ,co.remarks		
                ,CASE		
                	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'	
                	WHEN so.modified_by_id = 25 THEN 'Vinculum API'	
                	WHEN so.modified_by_id = 44306 THEN 'Delite'	
                	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'	
                	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'	
                	ELSE 'Others'END AS shipped_by	
                ,so.state_changed_on + time '5:30'AS shipped		
                ,ofd.state_changed_on + time '5:30'AS out_for_delivery		
                ,del.state_changed_on + time '5:30'AS delivered_date		
                ,iv.amount_paise/100*1.0 AS invoiced_amt		
                ,iv.invoiced_at		
                ,iv.wallet_amount		
                ,ph.number		
                ,us.name AS user_name		
                ,us.created_at AS registration_date		
                ,us.registration_source		
                ,areas.pincode
                --,op.promotion_id
                --,op.promotion_total_paise/100*1.0 as amount_discounted
                --,op.cash_back_total_paise/100*1.0 as cashback
                --,promo.coupon_code
                ,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order		
                --,CASE WHEN 		
                --	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1	
                --	THEN '1st Order' ELSE 'Not First Order'		
                --END AS new_flag_month	
                ,CASE 		
                	 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'		
                	 WHEN del.state_changed_on + time '5:30' 		
                	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'	
                	 WHEN del.state_changed_on + time '5:30' 		
                	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) 	
                	 THEN 'Delay'	
                	 ELSE 'Between Slot'		
                END AS delivery_flag
                ,oah_t.created_at + time '5:30' as timeout_at
                ,oah_a.created_at + time '5:30' as accepted_at
                ,oah_r.created_at + time '5:30' as rejected_at
                
                FROM		
                	(SELECT	
                	order_id	
                	,MAX(modified_by_id) modified_by_id	
                	,MAX(channel) channel	
                	,MAX(state_changed_on + time '5:30') AS confirmed_on	
                	FROM order_status_logs	
                	WHERE state_changed_on >'2025-01-31 18:30' AND to_state = 3	
                	GROUP BY order_id) AS osl	
                LEFT JOIN orders o ON o.id = osl.order_id		
                LEFT JOIN users ON users.id = osl.modified_by_id		
                LEFT JOIN warehouses w ON w.id = o.fulfillment_center_id
                LEFT JOIN warehouses wa ON wa.id = o.store_id
                LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id		
                LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders		
                LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders		
                LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery		
                LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery		
                LEFT JOIN invoices iv ON iv.order_id = osl.order_id		
                LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL		
                LEFT JOIN users us ON us.id = o.user_id 		
                LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id 		
                LEFT JOIN areas ON areas.id = addresses.area_id
                LEFT JOIN order_assignment_histories oah_t on oah_t.order_id = osl.order_id AND oah_t.action = 'time_out'
                LEFT JOIN order_assignment_histories oah_a on oah_a.order_id = osl.order_id AND oah_a.action = 'accept'
                LEFT JOIN order_assignment_histories oah_r on oah_r.order_id = osl.order_id AND oah_r.action = 'reject'
                --LEFT JOIN order_promotions op on osl.order_id = op.order_id
                
                ORDER BY osl.confirmed_on DESC) as order_Table
                LEFT JOIN 
                (SELECT od FROM
                (SELECT ord.*,
                ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number 
                FROM
                (
                SELECT 
                order_id as od
                ,MAX(u.created_at) as registered_at
                ,MAX(o.user_id) as user_id
                ,MAX(state_changed_on + time '5:30') as confirmed_on
                ,MAX(channel) as channel
                ,MAX(modified_by_id) as modified_by
                FROM order_status_logs osl 
                LEFT JOIN orders o on o.id = osl.order_id
                LEFT JOIN users u on u.id = o.user_id
                WHERE to_state = 3 and state_changed_on > '2022-12-31 18:30:00'
                GROUP BY od
                ) AS ord) 
                AS ordr
                WHERE order_number = 1
                AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
                --AND EXTRACT(MONTH FROM confirmed_on) = EXTRACT(MONTH FROM CURRENT_DATE)
                AND confirmed_on >= '2025-01-31 18:30:00'
                --AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE)
                ) as first_order
                on first_order.od = order_Table.order_id
                --LEFT JOIN promotions promo on promo.id = order_Table.promotion_id
                order by order_Table.confirmed_on DESC
                ''')
                
                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description] # Get column names from cursor description
                df = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
                logging.info(f'{len(records)} fetched from DB anmd saved in df')
                df = df[['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
                         'payment_method', 'new_flag', 'order_value', 'shipping_charge',
                         'detailed_status', 'out_for_delivery', 'confirmed_on', 'channel_name',
                         'shipped', 'fulfillment_center', 'slot_description', 'expected_delivery',
                         'modified_by', 'reason', 'remarks', 'cancelled_date',
                         'delivered_date', 'channel2', 'invoiced_amt', 'shipped_by',
                         'exp_delivery_start', 'exp_delivery_end', 'delivery_flag', 'number','user_name' ,'pincode','wallet_amount'
                         ,'order_action','actual_mapped_dc','registration_date']]
                logging.info('Data rearranged')

        except (Exception, psycopg2.Error) as error:
            logging.info(f'Error connecting to PostgreSQL: {error}')
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                logging.info("PostgreSQL connection is closed & Job Done")
        
        gsheet_name = 'Summary_Epharmacy' #This google sheet will be updated
        tab_name = 'C42_Combined' # This particular tab is to be updated
        
        #Updating In Summary_Epharmacy
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1etqrto99N3Tmv9Z-svwa4OtFjUewpP0KwMiflq36hrg") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f'{gsheet_name} sheet updated')
        write_df_to_gsheet(gsheet_name,tab_name,df)
        

        #Updating In cc_daily_dashboard
        gsheet_name = 'CC Daily Order Dashboard' #This google sheet will be updated
        tab_name = 'C42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1_ibELjBzTBBjKeNkgsIm5EzrDo5vamHZraOjMjDqNvI") #Key Of the google sheet - CC order dashboard
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f'{gsheet_name} sheet updated')
        write_df_to_gsheet(gsheet_name,tab_name,df)
        


        #Updating In Reorder_sheet
        gsheet_name = 'Reorder' #This google sheet will be updated
        tab_name = 'c42_current_month' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1WBLi5ShCGQ1US-x3OxPT2mXmz8sCn5UxiqtXJ16poBU") #Key Of the google sheet - CC order dashboard
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f'Current month tab in {gsheet_name} sheet updated')
        write_df_to_gsheet(gsheet_name,tab_name,df)
        
        
    
    except (Exception, psycopg2.Error) as error:
        logging.info(f"Error connecting to PostgreSQL: {error}")
    finally:# Close the database connection
        if connection:
            cursor.close()
            connection.close()
            logging.info("PostgreSQL connection is closed\n")

logging.info('Schedule the task to run every hour')
schedule.every().hour.at(":01").do(Order_summary_update)
#print(schedule.get_jobs())
while True:
    schedule.run_pending()
    time.sleep(5)

In [ ]:

# This Script Updates current month order in Summary sheet , CC Dashboar, Re order sheet, - Logging Configured
import schedule as schedule
import time
import logging

logging.basicConfig(
    filename="log_Order_summary_update.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def Order_summary_update():
    try:
        import csv
        import pandas as pd
        import numpy as np
        import psycopg2
        import pandas as pd
        import numpy as np
        import paramiko
        import re
        from sshtunnel import SSHTunnelForwarder
        import gspread
        from gspread_dataframe import set_with_dataframe
        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                        database='efrprod',
                        user='emamireaduser',
                        password='emamireadaccess'
                )
                cursor = connection.cursor()

                logging.info('Executing DB query.....')
                cursor.execute(
                '''
                SELECT order_Table.*
                ,CASE WHEN
                	od IS NULL THEN 'Not First Order'
                	ELSE '1st Order' END AS new_flag
                ,CASE
                 	WHEN timeout_at IS NOT NULL THEN 'timeout'
                	WHEN rejected_at IS NOT NULL THEN 'rejected'
                	WHEN accepted_at IS NOT NULL THEN 'accepted'
                	ELSE 'pending' end as order_action
                FROM
                (SELECT
                 osl.order_id
                ,o.user_id as created_by_id
                ,CASE
                	WHEN o.state =0 THEN 'cart'
                	WHEN o.state =1 THEN 'pre_checkout'
                	WHEN o.state =2 THEN 'Checkout'
                	WHEN o.state =6 THEN 'Delivered'
                	WHEN o.state =7 THEN 'Cancelled'
                	WHEN o.state =14  THEN 'Return_completed'
                	ELSE 'In-Progress'
                END AS status
                ,o.city_id
                ,o.delivery_remarks
                ,o.payment_method
                ,o.auto_completed
                ,o.order_total_paise*1.0/100 AS order_value
                ,o.shipping_total_paise*1.0/100 AS Shipping_charge
                ,o.doctor_names
                --,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id
                ,osl.confirmed_on
                ,CASE
                	WHEN osl.channel = 0 THEN 'Call Center'
                	WHEN osl.channel = 1 THEN 'Mobile'
                	ELSE 'Website'
                END as channel_name
                ,CASE
                	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
                	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                	WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                	WHEN osl.channel = 0 THEN 'Call Center'
                	WHEN osl.channel = 1 THEN 'Saathi App'
                	ELSE 'Website'
                END as channel2
                ,CASE
                	WHEN o.state =0 THEN 'cart'
                	WHEN o.state =1 THEN 'pre_checkout'
                	WHEN o.state =2 THEN 'Checkout'
                	WHEN o.state =3 THEN 'order_received'
                	WHEN o.state =4 THEN 'shipped'
                	WHEN o.state =5 THEN 'out_for_delivery'
                	WHEN o.state =6 THEN 'Delivered'
                	WHEN o.state =7 THEN 'Cancelled'
                	WHEN o.state =14 THEN 'Return_completed'
                	WHEN o.state =17 THEN 'rescheduled'
                		ELSE 'In-Progress' END AS detailed_status
                ,customer_id
                ,users.name AS modified_by
                ,w.code AS fulfillment_center
                ,Wa.code AS Actual_Mapped_Dc
                ,DS.slot_description
                ,DS.slot_date AS Expected_Delivery
                ,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start
                ,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end
                ,co.state_changed_on + time '5:30' as cancelled_date
                ,co.reason
                ,co.remarks
                ,CASE
                	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'
                	WHEN so.modified_by_id = 25 THEN 'Vinculum API'
                	WHEN so.modified_by_id = 44306 THEN 'Delite'
                	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'
                	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'
                	ELSE 'Others'END AS shipped_by
                ,so.state_changed_on + time '5:30'AS shipped
                ,ofd.state_changed_on + time '5:30'AS out_for_delivery
                ,del.state_changed_on + time '5:30'AS delivered_date
                ,iv.amount_paise/100*1.0 AS invoiced_amt
                ,iv.invoiced_at
                ,iv.wallet_amount
                ,ph.number
                ,us.name AS user_name
                ,us.created_at AS registration_date
                ,us.registration_source
                ,areas.pincode
                ,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order
                --,CASE WHEN
                --	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1
                --	THEN '1st Order' ELSE 'Not First Order'
                --END AS new_flag_month
                ,CASE
                	 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'
                	 WHEN del.state_changed_on + time '5:30'
                	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'
                	 WHEN del.state_changed_on + time '5:30'
                	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME)
                	 THEN 'Delay'
                	 ELSE 'Between Slot'
                END AS delivery_flag
                ,oah_t.created_at + time '5:30' as timeout_at
                ,oah_a.created_at + time '5:30' as accepted_at
                ,oah_r.created_at + time '5:30' as rejected_at

                FROM
                	(SELECT
                	order_id
                	,MAX(modified_by_id) modified_by_id
                	,MAX(channel) channel
                	,MAX(state_changed_on + time '5:30') AS confirmed_on
                	FROM order_status_logs
                	WHERE state_changed_on >'2024-12-31 18:30' AND to_state = 3
                	GROUP BY order_id) AS osl
                LEFT JOIN orders o ON o.id = osl.order_id
                LEFT JOIN users ON users.id = osl.modified_by_id
                LEFT JOIN warehouses w ON w.id = o.fulfillment_center_id
                LEFT JOIN warehouses wa ON wa.id = o.store_id
                LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id
                LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders
                LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders
                LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery
                LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery
                LEFT JOIN invoices iv ON iv.order_id = osl.order_id
                LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL
                LEFT JOIN users us ON us.id = o.user_id
                LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id
                LEFT JOIN areas ON areas.id = addresses.area_id
                LEFT JOIN order_assignment_histories oah_t on oah_t.order_id = osl.order_id AND oah_t.action = 'time_out'
                LEFT JOIN order_assignment_histories oah_a on oah_a.order_id = osl.order_id AND oah_a.action = 'accept'
                LEFT JOIN order_assignment_histories oah_r on oah_r.order_id = osl.order_id AND oah_r.action = 'reject'
                ORDER BY osl.confirmed_on DESC) as order_Table
                LEFT JOIN
                (SELECT od FROM
                (SELECT ord.*,
                ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number
                FROM
                (
                SELECT
                order_id as od
                ,MAX(u.created_at) as registered_at
                ,MAX(o.user_id) as user_id
                ,MAX(state_changed_on + time '5:30') as confirmed_on
                ,MAX(channel) as channel
                ,MAX(modified_by_id) as modified_by
                FROM order_status_logs osl
                LEFT JOIN orders o on o.id = osl.order_id
                LEFT JOIN users u on u.id = o.user_id
                WHERE to_state = 3 and state_changed_on > '2022-12-31 18:30:00'
                GROUP BY od
                ) AS ord)
                AS ordr
                WHERE order_number = 1
                AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
                --AND EXTRACT(MONTH FROM confirmed_on) = EXTRACT(MONTH FROM CURRENT_DATE)
                AND confirmed_on >= '2024-12-31 18:30:00'
                --AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE)
                ) as first_order
                on first_order.od = order_Table.order_id
                order by order_Table.confirmed_on DESC
                ''')

                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description] # Get column names from cursor description
                df = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
                logging.info(f'{len(records)} fetched from DB anmd saved in df')
                df = df[['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
                         'payment_method', 'new_flag', 'order_value', 'shipping_charge',
                         'detailed_status', 'out_for_delivery', 'confirmed_on', 'channel_name',
                         'shipped', 'fulfillment_center', 'slot_description', 'expected_delivery',
                         'modified_by', 'reason', 'remarks', 'cancelled_date',
                         'delivered_date', 'channel2', 'invoiced_amt', 'shipped_by',
                         'exp_delivery_start', 'exp_delivery_end', 'delivery_flag', 'number','user_name' ,'pincode','wallet_amount'
                         ,'order_action','actual_mapped_dc']]
                logging.info('Data rearranged')

        except (Exception, psycopg2.Error) as error:
            logging.info(f'Error connecting to PostgreSQL: {error}')
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                logging.info("PostgreSQL connection is closed & Job Done")

        gsheet_name = 'Summary_Epharmacy' #This google sheet will be updated
        tab_name = 'C42_Combined' # This particular tab is to be updated

        #Updating In Summary_Epharmacy
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1etqrto99N3Tmv9Z-svwa4OtFjUewpP0KwMiflq36hrg") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f'{gsheet_name} sheet updated')
        write_df_to_gsheet(gsheet_name,tab_name,df)


        #Updating In cc_daily_dashboard
        gsheet_name = 'CC Daily Order Dashboard' #This google sheet will be updated
        tab_name = 'C42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1_ibELjBzTBBjKeNkgsIm5EzrDo5vamHZraOjMjDqNvI") #Key Of the google sheet - CC order dashboard
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f'{gsheet_name} sheet updated')
        write_df_to_gsheet(gsheet_name,tab_name,df)



        #Updating In Reorder_sheet
        gsheet_name = 'Reorder' #This google sheet will be updated
        tab_name = 'c42_current_month' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1WBLi5ShCGQ1US-x3OxPT2mXmz8sCn5UxiqtXJ16poBU") #Key Of the google sheet - CC order dashboard
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f'{gsheet_name} sheet updated')
        write_df_to_gsheet(gsheet_name,tab_name,df)



    except (Exception, psycopg2.Error) as error:
        logging.info(f"Error connecting to PostgreSQL: {error}")
    finally:# Close the database connection
        if connection:
            cursor.close()
            connection.close()
            logging.info("PostgreSQL connection is closed\n")

logging.info('Schedule the task to run every hour')
schedule.every().hour.at(":01").do(Order_summary_update)
#print(schedule.get_jobs())
while True:
    schedule.run_pending()
    time.sleep(5)

In [ ]:
# This Script Updates current month order in Summary sheet , CC Dashboar, Re order sheet, - Logging Configured
import schedule as schedule
import time
import logging

logging.basicConfig(
    filename="log_Order_summary_update.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def Order_summary_update():
    try:
        import csv
        import pandas as pd
        import numpy as np
        import psycopg2
        import pandas as pd
        import numpy as np
        import paramiko
        import re
        from sshtunnel import SSHTunnelForwarder
        import gspread
        from gspread_dataframe import set_with_dataframe
        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                        database='efrprod',
                        user='emamireaduser',
                        password='emamireadaccess'
                )
                cursor = connection.cursor()

                logging.info('Executing DB query.....')
                cursor.execute(
                '''
                SELECT order_Table.*
                ,CASE WHEN
                	od IS NULL THEN 'Not First Order'
                	ELSE '1st Order' END AS new_flag
                ,CASE
                 	WHEN timeout_at IS NOT NULL THEN 'timeout'
                	WHEN rejected_at IS NOT NULL THEN 'rejected'
                	WHEN accepted_at IS NOT NULL THEN 'accepted'
                	ELSE 'pending' end as order_action
                FROM
                (SELECT
                 osl.order_id
                ,o.user_id as created_by_id
                ,CASE
                	WHEN o.state =0 THEN 'cart'
                	WHEN o.state =1 THEN 'pre_checkout'
                	WHEN o.state =2 THEN 'Checkout'
                	WHEN o.state =6 THEN 'Delivered'
                	WHEN o.state =7 THEN 'Cancelled'
                	WHEN o.state =14  THEN 'Return_completed'
                	ELSE 'In-Progress'
                END AS status
                ,o.city_id
                ,o.delivery_remarks
                ,o.payment_method
                ,o.auto_completed
                ,o.order_total_paise*1.0/100 AS order_value
                ,o.shipping_total_paise*1.0/100 AS Shipping_charge
                ,o.doctor_names
                --,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id
                ,osl.confirmed_on
                ,CASE
                	WHEN osl.channel = 0 THEN 'Call Center'
                	WHEN osl.channel = 1 THEN 'Mobile'
                	ELSE 'Website'
                END as channel_name
                ,CASE
                	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
                	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                	WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                	WHEN osl.channel = 0 THEN 'Call Center'
                	WHEN osl.channel = 1 THEN 'Saathi App'
                	ELSE 'Website'
                END as channel2
                ,CASE
                	WHEN o.state =0 THEN 'cart'
                	WHEN o.state =1 THEN 'pre_checkout'
                	WHEN o.state =2 THEN 'Checkout'
                	WHEN o.state =3 THEN 'order_received'
                	WHEN o.state =4 THEN 'shipped'
                	WHEN o.state =5 THEN 'out_for_delivery'
                	WHEN o.state =6 THEN 'Delivered'
                	WHEN o.state =7 THEN 'Cancelled'
                	WHEN o.state =14 THEN 'Return_completed'
                	WHEN o.state =17 THEN 'rescheduled'
                		ELSE 'In-Progress' END AS detailed_status
                ,customer_id
                ,users.name AS modified_by
                ,w.code AS fulfillment_center
                ,Wa.code AS Actual_Mapped_Dc
                ,DS.slot_description
                ,DS.slot_date AS Expected_Delivery
                ,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start
                ,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end
                ,co.state_changed_on + time '5:30' as cancelled_date
                ,co.reason
                ,co.remarks
                ,CASE
                	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'
                	WHEN so.modified_by_id = 25 THEN 'Vinculum API'
                	WHEN so.modified_by_id = 44306 THEN 'Delite'
                	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'
                	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'
                	ELSE 'Others'END AS shipped_by
                ,so.state_changed_on + time '5:30'AS shipped
                ,ofd.state_changed_on + time '5:30'AS out_for_delivery
                ,del.state_changed_on + time '5:30'AS delivered_date
                ,iv.amount_paise/100*1.0 AS invoiced_amt
                ,iv.invoiced_at
                ,iv.wallet_amount
                ,ph.number
                ,us.name AS user_name
                ,us.created_at AS registration_date
                ,us.registration_source
                ,areas.pincode
                ,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order
                --,CASE WHEN
                --	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1
                --	THEN '1st Order' ELSE 'Not First Order'
                --END AS new_flag_month
                ,CASE
                	 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'
                	 WHEN del.state_changed_on + time '5:30'
                	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'
                	 WHEN del.state_changed_on + time '5:30'
                	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME)
                	 THEN 'Delay'
                	 ELSE 'Between Slot'
                END AS delivery_flag
                ,oah_t.created_at + time '5:30' as timeout_at
                ,oah_a.created_at + time '5:30' as accepted_at
                ,oah_r.created_at + time '5:30' as rejected_at

                FROM
                	(SELECT
                	order_id
                	,MAX(modified_by_id) modified_by_id
                	,MAX(channel) channel
                	,MAX(state_changed_on + time '5:30') AS confirmed_on
                	FROM order_status_logs
                	WHERE state_changed_on >'2024-12-31 18:30' AND to_state = 3
                	GROUP BY order_id) AS osl
                LEFT JOIN orders o ON o.id = osl.order_id
                LEFT JOIN users ON users.id = osl.modified_by_id
                LEFT JOIN warehouses w ON w.id = o.fulfillment_center_id
                LEFT JOIN warehouses wa ON wa.id = o.store_id
                LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id
                LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders
                LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders
                LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery
                LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery
                LEFT JOIN invoices iv ON iv.order_id = osl.order_id
                LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL
                LEFT JOIN users us ON us.id = o.user_id
                LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id
                LEFT JOIN areas ON areas.id = addresses.area_id
                LEFT JOIN order_assignment_histories oah_t on oah_t.order_id = osl.order_id AND oah_t.action = 'time_out'
                LEFT JOIN order_assignment_histories oah_a on oah_a.order_id = osl.order_id AND oah_a.action = 'accept'
                LEFT JOIN order_assignment_histories oah_r on oah_r.order_id = osl.order_id AND oah_r.action = 'reject'
                ORDER BY osl.confirmed_on DESC) as order_Table
                LEFT JOIN
                (SELECT od FROM
                (SELECT ord.*,
                ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number
                FROM
                (
                SELECT
                order_id as od
                ,MAX(u.created_at) as registered_at
                ,MAX(o.user_id) as user_id
                ,MAX(state_changed_on + time '5:30') as confirmed_on
                ,MAX(channel) as channel
                ,MAX(modified_by_id) as modified_by
                FROM order_status_logs osl
                LEFT JOIN orders o on o.id = osl.order_id
                LEFT JOIN users u on u.id = o.user_id
                WHERE to_state = 3 and state_changed_on > '2022-12-31 18:30:00'
                GROUP BY od
                ) AS ord)
                AS ordr
                WHERE order_number = 1
                AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
                --AND EXTRACT(MONTH FROM confirmed_on) = EXTRACT(MONTH FROM CURRENT_DATE)
                AND confirmed_on >= '2024-12-31 18:30:00'
                --AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE)
                ) as first_order
                on first_order.od = order_Table.order_id
                order by order_Table.confirmed_on DESC
                ''')

                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description] # Get column names from cursor description
                df = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
                logging.info(f'{len(records)} fetched from DB anmd saved in df')
                df = df[['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
                         'payment_method', 'new_flag', 'order_value', 'shipping_charge',
                         'detailed_status', 'out_for_delivery', 'confirmed_on', 'channel_name',
                         'shipped', 'fulfillment_center', 'slot_description', 'expected_delivery',
                         'modified_by', 'reason', 'remarks', 'cancelled_date',
                         'delivered_date', 'channel2', 'invoiced_amt', 'shipped_by',
                         'exp_delivery_start', 'exp_delivery_end', 'delivery_flag', 'number','user_name' ,'pincode','wallet_amount'
                         ,'order_action','actual_mapped_dc']]
                logging.info('Data rearranged')

        except (Exception, psycopg2.Error) as error:
            logging.info(f'Error connecting to PostgreSQL: {error}')
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                logging.info("PostgreSQL connection is closed & Job Done")

        gsheet_name = 'Summary_Epharmacy' #This google sheet will be updated
        tab_name = 'C42_Combined' # This particular tab is to be updated

        #Updating In Summary_Epharmacy
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1etqrto99N3Tmv9Z-svwa4OtFjUewpP0KwMiflq36hrg") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f'{gsheet_name} sheet updated')
        write_df_to_gsheet(gsheet_name,tab_name,df)


        #Updating In cc_daily_dashboard
        gsheet_name = 'CC Daily Order Dashboard' #This google sheet will be updated
        tab_name = 'C42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1_ibELjBzTBBjKeNkgsIm5EzrDo5vamHZraOjMjDqNvI") #Key Of the google sheet - CC order dashboard
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f'{gsheet_name} sheet updated')
        write_df_to_gsheet(gsheet_name,tab_name,df)



        #Updating In Reorder_sheet
        gsheet_name = 'Reorder' #This google sheet will be updated
        tab_name = 'c42_current_month' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1WBLi5ShCGQ1US-x3OxPT2mXmz8sCn5UxiqtXJ16poBU") #Key Of the google sheet - CC order dashboard
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f'{gsheet_name} sheet updated')
        write_df_to_gsheet(gsheet_name,tab_name,df)



    except (Exception, psycopg2.Error) as error:
        logging.info(f"Error connecting to PostgreSQL: {error}")
    finally:# Close the database connection
        if connection:
            cursor.close()
            connection.close()
            logging.info("PostgreSQL connection is closed\n")

logging.info('Schedule the task to run every hour')
schedule.every().hour.at(":57").do(Order_summary_update)
#print(schedule.get_jobs())
while True:
    schedule.run_pending()
    time.sleep(5)

In [ ]:
#This script updates orders data - updates : added order_action and actual mapped dc and Fulfillment center

import schedule as schedule
import time
def Order_summary_update():
    try:
        import csv
        import pandas as pd
        import numpy as np
        import psycopg2
        import pandas as pd
        import numpy as np
        import paramiko
        import re
        from sshtunnel import SSHTunnelForwarder
        import gspread
        from gspread_dataframe import set_with_dataframe
        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                        database='efrprod',
                        user='emamireaduser',
                        password='emamireadaccess'
                )
                cursor = connection.cursor()

                # Perform database operations here
                cursor.execute(
                '''
                SELECT order_Table.*
                ,CASE WHEN
                	od IS NULL THEN 'Not First Order'
                	ELSE '1st Order' END AS new_flag
                ,CASE
                 	WHEN timeout_at IS NOT NULL THEN 'timeout'
                	WHEN rejected_at IS NOT NULL THEN 'rejected'
                	WHEN accepted_at IS NOT NULL THEN 'accepted'
                	ELSE 'pending' end as order_action
                FROM
                (SELECT
                 osl.order_id
                ,o.user_id as created_by_id
                ,CASE
                	WHEN o.state =0 THEN 'cart'
                	WHEN o.state =1 THEN 'pre_checkout'
                	WHEN o.state =2 THEN 'Checkout'
                	WHEN o.state =6 THEN 'Delivered'
                	WHEN o.state =7 THEN 'Cancelled'
                	WHEN o.state =14  THEN 'Return_completed'
                	ELSE 'In-Progress'
                END AS status
                ,o.city_id
                ,o.delivery_remarks
                ,o.payment_method
                ,o.auto_completed
                ,o.order_total_paise*1.0/100 AS order_value
                ,o.shipping_total_paise*1.0/100 AS Shipping_charge
                ,o.doctor_names
                --,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id
                ,osl.confirmed_on
                ,CASE
                	WHEN osl.channel = 0 THEN 'Call Center'
                	WHEN osl.channel = 1 THEN 'Mobile'
                	ELSE 'Website'
                END as channel_name
                ,CASE
                	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
                	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                	WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                	WHEN osl.channel = 0 THEN 'Call Center'
                	WHEN osl.channel = 1 THEN 'Saathi App'
                	ELSE 'Website'
                END as channel2
                ,CASE
                	WHEN o.state =0 THEN 'cart'
                	WHEN o.state =1 THEN 'pre_checkout'
                	WHEN o.state =2 THEN 'Checkout'
                	WHEN o.state =3 THEN 'order_received'
                	WHEN o.state =4 THEN 'shipped'
                	WHEN o.state =5 THEN 'out_for_delivery'
                	WHEN o.state =6 THEN 'Delivered'
                	WHEN o.state =7 THEN 'Cancelled'
                	WHEN o.state =14 THEN 'Return_completed'
                	WHEN o.state =17 THEN 'rescheduled'
                		ELSE 'In-Progress' END AS detailed_status
                ,customer_id
                ,users.name AS modified_by
                ,w.code AS fulfillment_center
                ,Wa.code AS Actual_Mapped_Dc
                ,DS.slot_description
                ,DS.slot_date AS Expected_Delivery
                ,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start
                ,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end
                ,co.state_changed_on + time '5:30' as cancelled_date
                ,co.reason
                ,co.remarks
                ,CASE
                	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'
                	WHEN so.modified_by_id = 25 THEN 'Vinculum API'
                	WHEN so.modified_by_id = 44306 THEN 'Delite'
                	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'
                	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'
                	ELSE 'Others'END AS shipped_by
                ,so.state_changed_on + time '5:30'AS shipped
                ,ofd.state_changed_on + time '5:30'AS out_for_delivery
                ,del.state_changed_on + time '5:30'AS delivered_date
                ,iv.amount_paise/100*1.0 AS invoiced_amt
                ,iv.invoiced_at
                ,iv.wallet_amount
                ,ph.number
                ,us.name AS user_name
                ,us.created_at AS registration_date
                ,us.registration_source
                ,areas.pincode
                ,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order
                --,CASE WHEN
                --	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1
                --	THEN '1st Order' ELSE 'Not First Order'
                --END AS new_flag_month
                ,CASE
                	 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'
                	 WHEN del.state_changed_on + time '5:30'
                	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'
                	 WHEN del.state_changed_on + time '5:30'
                	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME)
                	 THEN 'Delay'
                	 ELSE 'Between Slot'
                END AS delivery_flag
                ,oah_t.created_at + time '5:30' as timeout_at
                ,oah_a.created_at + time '5:30' as accepted_at
                ,oah_r.created_at + time '5:30' as rejected_at

                FROM
                	(SELECT
                	order_id
                	,MAX(modified_by_id) modified_by_id
                	,MAX(channel) channel
                	,MAX(state_changed_on + time '5:30') AS confirmed_on
                	FROM order_status_logs
                	WHERE state_changed_on >'2024-12-31 18:30' AND to_state = 3
                	GROUP BY order_id) AS osl
                LEFT JOIN orders o ON o.id = osl.order_id
                LEFT JOIN users ON users.id = osl.modified_by_id
                LEFT JOIN warehouses w ON w.id = o.fulfillment_center_id
                LEFT JOIN warehouses wa ON wa.id = o.store_id
                LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id
                LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders
                LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders
                LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery
                LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery
                LEFT JOIN invoices iv ON iv.order_id = osl.order_id
                LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL
                LEFT JOIN users us ON us.id = o.user_id
                LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id
                LEFT JOIN areas ON areas.id = addresses.area_id
                LEFT JOIN order_assignment_histories oah_t on oah_t.order_id = osl.order_id AND oah_t.action = 'time_out'
                LEFT JOIN order_assignment_histories oah_a on oah_a.order_id = osl.order_id AND oah_a.action = 'accept'
                LEFT JOIN order_assignment_histories oah_r on oah_r.order_id = osl.order_id AND oah_r.action = 'reject'
                ORDER BY osl.confirmed_on DESC) as order_Table
                LEFT JOIN
                (SELECT od FROM
                (SELECT ord.*,
                ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number
                FROM
                (
                SELECT
                order_id as od
                ,MAX(u.created_at) as registered_at
                ,MAX(o.user_id) as user_id
                ,MAX(state_changed_on + time '5:30') as confirmed_on
                ,MAX(channel) as channel
                ,MAX(modified_by_id) as modified_by
                FROM order_status_logs osl
                LEFT JOIN orders o on o.id = osl.order_id
                LEFT JOIN users u on u.id = o.user_id
                WHERE to_state = 3 and state_changed_on > '2022-12-31 18:30:00'
                GROUP BY od
                ) AS ord)
                AS ordr
                WHERE order_number = 1
                AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
                --AND EXTRACT(MONTH FROM confirmed_on) = EXTRACT(MONTH FROM CURRENT_DATE)
                AND confirmed_on >= '2024-12-31 18:30:00'
                --AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE)
                ) as first_order
                on first_order.od = order_Table.order_id
                order by order_Table.confirmed_on DESC
                ''')

                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description] # Get column names from cursor description
                df = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
                df = df[['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
                         'payment_method', 'new_flag', 'order_value', 'shipping_charge',
                         'detailed_status', 'out_for_delivery', 'confirmed_on', 'channel_name',
                         'shipped', 'fulfillment_center', 'slot_description', 'expected_delivery',
                         'modified_by', 'reason', 'remarks', 'cancelled_date',
                         'delivered_date', 'channel2', 'invoiced_amt', 'shipped_by',
                         'exp_delivery_start', 'exp_delivery_end', 'delivery_flag', 'number','user_name' ,'pincode','wallet_amount'
                         ,'order_action','actual_mapped_dc']]
                print("Data fetched and saved to df")

        except (Exception, psycopg2.Error) as error:
            print("Error connecting to PostgreSQL:", error)
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                print("PostgreSQL connection is closed")

        gsheet_name = 'Summary_Epharmacy' #This google sheet will be updated
        tab_name = 'C42_Combined' # This particular tab is to be updated

        #Updating In Summary_Epharmacy
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1etqrto99N3Tmv9Z-svwa4OtFjUewpP0KwMiflq36hrg") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
        write_df_to_gsheet(gsheet_name,tab_name,df)
        df.to_csv('c42_f.csv')
        print("Google sheet Updated_at", pd.Timestamp.now())

        #Updating In cc_daily_dashboard
        gsheet_name = 'CC Daily Order Dashboard' #This google sheet will be updated
        tab_name = 'C42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1_ibELjBzTBBjKeNkgsIm5EzrDo5vamHZraOjMjDqNvI") #Key Of the google sheet - CC order dashboard
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
        write_df_to_gsheet(gsheet_name,tab_name,df)
        print("CC_Google_sheet Updated_at", pd.Timestamp.now())


        #Updating In Reorder_sheet
        gsheet_name = 'Reorder' #This google sheet will be updated
        tab_name = 'c42_current_month' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1WBLi5ShCGQ1US-x3OxPT2mXmz8sCn5UxiqtXJ16poBU") #Key Of the google sheet - CC order dashboard
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
        write_df_to_gsheet(gsheet_name,tab_name,df)
        print("Reorder_sheet Updated_at", pd.Timestamp.now())


    except (Exception, psycopg2.Error) as error:
        print("Error connecting to PostgreSQL:", error)
    finally:# Close the database connection
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

# Schedule the task to run every minute
schedule.every().hour.at(":29").do(Order_summary_update)
#print(schedule.get_jobs())
while True:
    schedule.run_pending()
    time.sleep(5)

In [ ]:
#This script updates old orders data - working till Jan'2025
import schedule as schedule
import time
def Order_summary_update():
    try:
        import csv
        import pandas as pd
        import numpy as np
        import psycopg2
        import pandas as pd
        import numpy as np
        import paramiko
        import re
        from sshtunnel import SSHTunnelForwarder
        import gspread
        from gspread_dataframe import set_with_dataframe
        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                        database='efrprod',
                        user='emamireaduser',
                        password='emamireadaccess'
                )
                cursor = connection.cursor()

                # Perform database operations here
                cursor.execute(
                '''
                SELECT order_Table.*
                ,CASE WHEN
                	od IS NULL THEN 'Not First Order'
                	ELSE '1st Order' END AS new_flag
                FROM
                (SELECT
                 osl.order_id
                ,o.user_id as created_by_id
                ,CASE
                	WHEN o.state =0 THEN 'cart'
                	WHEN o.state =1 THEN 'pre_checkout'
                	WHEN o.state =2 THEN 'Checkout'
                	WHEN o.state =6 THEN 'Delivered'
                	WHEN o.state =7 THEN 'Cancelled'
                	WHEN o.state =14  THEN 'Return_completed'
                	ELSE 'In-Progress'
                END AS status
                ,o.city_id
                ,o.delivery_remarks
                ,o.payment_method
                ,o.auto_completed
                ,o.order_total_paise*1.0/100 AS order_value
                ,o.shipping_total_paise*1.0/100 AS Shipping_charge
                ,o.doctor_names
                --,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id
                ,osl.confirmed_on
                ,CASE
                	WHEN osl.channel = 0 THEN 'Call Center'
                	WHEN osl.channel = 1 THEN 'Mobile'
                	ELSE 'Website'
                END as channel_name
                ,CASE
                	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
                	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                	WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                	WHEN osl.channel = 0 THEN 'Call Center'
                	WHEN osl.channel = 1 THEN 'Saathi App'
                	ELSE 'Website'
                END as channel2
                ,CASE
                	WHEN o.state =0 THEN 'cart'
                	WHEN o.state =1 THEN 'pre_checkout'
                	WHEN o.state =2 THEN 'Checkout'
                	WHEN o.state =3 THEN 'order_received'
                	WHEN o.state =4 THEN 'shipped'
                	WHEN o.state =5 THEN 'out_for_delivery'
                	WHEN o.state =6 THEN 'Delivered'
                	WHEN o.state =7 THEN 'Cancelled'
                	WHEN o.state =14 THEN 'Return_completed'
                	WHEN o.state =17 THEN 'rescheduled'
                		ELSE 'In-Progress' END AS detailed_status
                ,customer_id
                ,users.name AS modified_by
                ,w.code AS dc_code
                ,DS.slot_description
                ,DS.slot_date AS Expected_Delivery
                ,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start
                ,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end
                ,co.state_changed_on + time '5:30' as cancelled_date
                ,co.reason
                ,co.remarks
                ,CASE
                	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'
                	WHEN so.modified_by_id = 25 THEN 'Vinculum API'
                	WHEN so.modified_by_id = 44306 THEN 'Delite'
                	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'
                	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'
                	ELSE 'Others'END AS shipped_by
                ,so.state_changed_on + time '5:30'AS shipped
                ,ofd.state_changed_on + time '5:30'AS out_for_delivery
                ,del.state_changed_on + time '5:30'AS delivered_date
                ,iv.amount_paise/100*1.0 AS invoiced_amt
                ,iv.invoiced_at
                ,iv.wallet_amount
                ,ph.number
                ,us.name AS user_name
                ,us.created_at AS registration_date
                ,us.registration_source
                ,areas.pincode
                ,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order
                --,CASE WHEN
                --	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1
                --	THEN '1st Order' ELSE 'Not First Order'
                --END AS new_flag_month
                ,CASE
                	 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'
                	 WHEN del.state_changed_on + time '5:30'
                	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'
                	 WHEN del.state_changed_on + time '5:30'
                	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME)
                	 THEN 'Delay'
                	 ELSE 'Between Slot'
                END AS delivery_flag
                FROM
                	(SELECT
                	order_id
                	,MAX(modified_by_id) modified_by_id
                	,MAX(channel) channel
                	,MAX(state_changed_on + time '5:30') AS confirmed_on
                	FROM order_status_logs
                	WHERE state_changed_on >'2024-12-31 18:30' AND to_state = 3
                	GROUP BY order_id) AS osl
                LEFT JOIN orders o ON o.id = osl.order_id
                LEFT JOIN users ON users.id = osl.modified_by_id
                LEFT JOIN warehouses w ON w.id = o.store_id
                LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id
                LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders
                LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders
                LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery
                LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery
                LEFT JOIN invoices iv ON iv.order_id = osl.order_id
                LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL
                LEFT JOIN users us ON us.id = o.user_id
                LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id
                LEFT JOIN areas ON areas.id = addresses.area_id
                ORDER BY osl.confirmed_on DESC) as order_Table
                LEFT JOIN
                (SELECT od FROM
                (SELECT ord.*,
                ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number
                FROM
                (
                SELECT
                order_id as od
                ,MAX(u.created_at) as registered_at
                ,MAX(o.user_id) as user_id
                ,MAX(state_changed_on + time '5:30') as confirmed_on
                ,MAX(channel) as channel
                ,MAX(modified_by_id) as modified_by
                FROM order_status_logs osl
                LEFT JOIN orders o on o.id = osl.order_id
                LEFT JOIN users u on u.id = o.user_id
                WHERE to_state = 3 and state_changed_on > '2023-01-31 18:30:00'
                GROUP BY od
                ) AS ord)
                AS ordr
                WHERE order_number = 1
                AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
                --AND EXTRACT(MONTH FROM confirmed_on) = EXTRACT(MONTH FROM CURRENT_DATE)
                AND confirmed_on >= '2024-12-31 18:30:00'
                --AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE)
                ) as first_order
                on first_order.od = order_Table.order_id
                order by order_Table.confirmed_on DESC
                ''')

                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description] # Get column names from cursor description
                df = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
                df = df[['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
                 'payment_method', 'new_flag', 'order_value', 'shipping_charge',
                 'detailed_status', 'out_for_delivery', 'confirmed_on', 'channel_name',
                 'shipped', 'dc_code', 'slot_description', 'expected_delivery',
                 'modified_by', 'reason', 'remarks', 'cancelled_date',
                 'delivered_date', 'channel2', 'invoiced_amt', 'shipped_by',
                 'exp_delivery_start', 'exp_delivery_end', 'delivery_flag', 'number', 'user_name','pincode','registration_date']]
                print("Data fetched and saved to df")

        except (Exception, psycopg2.Error) as error:
            print("Error connecting to PostgreSQL:", error)
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                print("PostgreSQL connection is closed")

        gsheet_name = 'Summary_Epharmacy' #This google sheet will be updated
        tab_name = 'C42_Combined' # This particular tab is to be updated

        #Updating In Summary_Epharmacy
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1etqrto99N3Tmv9Z-svwa4OtFjUewpP0KwMiflq36hrg") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
        write_df_to_gsheet(gsheet_name,tab_name,df)
        df.to_csv('c42_f.csv')
        print("Google sheet Updated_at", pd.Timestamp.now())

        #Updating In cc_daily_dashboard
        gsheet_name = 'CC Daily Order Dashboard' #This google sheet will be updated
        tab_name = 'C42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1_ibELjBzTBBjKeNkgsIm5EzrDo5vamHZraOjMjDqNvI") #Key Of the google sheet - CC order dashboard
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
        write_df_to_gsheet(gsheet_name,tab_name,df)
        print("CC_Google_sheet Updated_at", pd.Timestamp.now())


        #Updating In Reorder_sheet
        gsheet_name = 'Reorder' #This google sheet will be updated
        tab_name = 'c42_current_month' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1WBLi5ShCGQ1US-x3OxPT2mXmz8sCn5UxiqtXJ16poBU") #Key Of the google sheet - CC order dashboard
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
        write_df_to_gsheet(gsheet_name,tab_name,df)
        print("Reorder_sheet Updated_at", pd.Timestamp.now())


    except (Exception, psycopg2.Error) as error:
        print("Error connecting to PostgreSQL:", error)
    finally:# Close the database connection
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

# Schedule the task to run every minute
schedule.every().hour.at(":00").do(Order_summary_update)
#print(schedule.get_jobs())
while True:
    schedule.run_pending()
    time.sleep(5)

ModuleNotFoundError: No module named 'schedule'

# Sales report Backup At : 5th Feb2025

In [ ]:
# Sale update in CC_Dashboard- Added logging and threading
# Updated_at : 5th March'25 : to change date

import schedule
import time
import threading
import pandas as pd
import logging
from sshtunnel import SSHTunnelForwarder
import psycopg2
import gspread
from gspread_dataframe import set_with_dataframe

# Configure logging
logging.basicConfig(
    filename="log_sales.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

job_running = threading.Lock()

def line_items_sales():
    if not job_running.acquire(blocking=False):
        logging.warning("Job is already running, skipping this execution.")
        return

    try:
        logging.info("Job started.")
        connection = None  # Initialize connection to None at the start
        cursor = None  # Initialize cursor as None as well
        
        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                    host='127.0.0.1',
                    port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                    database='efrprod',
                    user='emamireaduser',
                    password='emamireadaccess'
                )
                cursor = connection.cursor()
                
                logging.info("Fetching item-wise sales data")
                cursor.execute('''
                SELECT
                line_items.order_id as ord_id,	
                line_items.variant_id,
                products.name as product_name,	
                categories.name as category,
                line_items.quantity,	
                line_items.mrp_paise * 1.0 / 100 as MRP,	
                line_items.sales_price_paise * 1.0 / 100 as Sales_price,	
                line_items.total_paise * 1.0 / 100 as Total_Price,	
                line_items.discount_amount_paise * 1.0 / 100 as Discount,
                --osl.modified_by_id,
                users.name as modified_by,
                CASE 
                    WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
                    WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                    WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                    WHEN osl.channel = 0 THEN 'Call Center'
                    WHEN osl.channel = 1 THEN 'Saathi_App'
                    ELSE 'Website'
                END as channel,
                osl.state_changed_on + time '5:30' as confirmed_on,
                CASE 
                    WHEN o.state = 0 THEN 'cart'
                    WHEN o.state = 1 THEN 'pre_checkout'
                    WHEN o.state = 2 THEN 'Checkout'
                    WHEN o.state = 6 THEN 'Delivered'
                    WHEN o.state = 7 THEN 'Cancelled'
                    WHEN o.state = 14 THEN 'Return_completed'
                    ELSE 'In-Progress' 
                END as status,
                dc.dc_code,
                ip.property_value as d_profile,
                ip2.property_value as actute_chronic
                FROM 
                line_items 
                LEFT JOIN variants ON line_items.variant_id = variants.id	
                LEFT JOIN products ON variants.product_id = products.id	
                LEFT JOIN product_types ON products.product_type_id = product_types.id	
                LEFT JOIN categories ON categories.id = product_types.category_id
                LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
                LEFT JOIN orders o ON o.id = osl.order_id
                LEFT JOIN distribution_centers dc ON dc.id = o.store_id
                LEFT JOIN item_properties ip ON ip.item_id = line_items.variant_id AND ip.property_id IN ('1351')
                LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
                LEFT JOIN users on users.id = osl.modified_by_id
                WHERE 
                line_items.order_id IN ( 
                    SELECT order_id 
                    FROM order_status_logs as ols
                    LEFT JOIN orders ON orders.id = ols.order_id
                    WHERE 
                        state_changed_on >= '2025-11-30 18:30:00.000000' 
                        AND to_state = 3
                        AND city_id = 13
                )
                AND osl.to_state = 3
                ORDER BY confirmed_on DESC
                ''')
                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]  # Get column names from cursor description
                sales = pd.DataFrame(records, columns=columns)  # Convert records to DataFrame
                
                logging.info(f"Fetched {len(records)} records from the database.")

        except (Exception, psycopg2.Error) as error:
            logging.error(f"Error connecting to PostgreSQL: {error}")
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                logging.info("PostgreSQL connection is closed.")

        # Updating data into Google Sheet
        gsheet_name = 'CC Order Dashboard'
        tab_name = 'Sales'

        def write_df_to_gsheet(gsheet_name, tab_name, sales):
            try:
                gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
                sh = gc.open_by_key("1_ibELjBzTBBjKeNkgsIm5EzrDo5vamHZraOjMjDqNvI")  # Key of the Google Sheet
                worksheet = sh.worksheet(tab_name)
                set_with_dataframe(worksheet, sales)
                logging.info(f"Sales data successfully updated in {gsheet_name} Sheet.")
            except Exception as e:
                logging.error(f"Error updating Google Sheet: {e}")

        write_df_to_gsheet(gsheet_name, tab_name, sales)
    except Exception as e:
        logging.error(f"Unexpected error in job execution: {e}")
    finally:
        job_running.release()
        logging.info("Job finished.\n")

# Schedule the job to run every hour at minute 02
schedule.every().hour.at(":03").do(line_items_sales)
logging.info("Job scheduled to run every hour.")

# Main loop to keep the scheduler running
while True:
    schedule.run_pending()
    time.sleep(60)  # Check for scheduled jobs every 60 seconds


In [ ]:
# Sale update in CC_Dashboard- Added logging and threading : Backup At : 5th Feb2025
import schedule
import time
import threading
import pandas as pd
import logging
from sshtunnel import SSHTunnelForwarder
import psycopg2
import gspread
from gspread_dataframe import set_with_dataframe

# Configure logging
logging.basicConfig(
    filename="log_sales.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

job_running = threading.Lock()

def line_items_sales():
    if not job_running.acquire(blocking=False):
        logging.warning("Job is already running, skipping this execution.")
        return

    try:
        logging.info("Job started.")
        connection = None  # Initialize connection to None at the start
        cursor = None  # Initialize cursor as None as well

        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                    host='127.0.0.1',
                    port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                    database='efrprod',
                    user='emamireaduser',
                    password='emamireadaccess'
                )
                cursor = connection.cursor()

                # Fetching item-wise sales data
                cursor.execute('''
                SELECT
                line_items.order_id as ord_id,
                line_items.variant_id,
                products.name as product_name,
                categories.name as category,
                line_items.quantity,
                line_items.mrp_paise * 1.0 / 100 as MRP,
                line_items.sales_price_paise * 1.0 / 100 as Sales_price,
                line_items.total_paise * 1.0 / 100 as Total_Price,
                line_items.discount_amount_paise * 1.0 / 100 as Discount,
                --osl.modified_by_id,
                users.name as modified_by,
                CASE
                    WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
                    WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                    WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                    WHEN osl.channel = 0 THEN 'Call Center'
                    WHEN osl.channel = 1 THEN 'Saathi_App'
                    ELSE 'Website'
                END as channel,
                osl.state_changed_on + time '5:30' as confirmed_on,
                CASE
                    WHEN o.state = 0 THEN 'cart'
                    WHEN o.state = 1 THEN 'pre_checkout'
                    WHEN o.state = 2 THEN 'Checkout'
                    WHEN o.state = 6 THEN 'Delivered'
                    WHEN o.state = 7 THEN 'Cancelled'
                    WHEN o.state = 14 THEN 'Return_completed'
                    ELSE 'In-Progress'
                END as status,
                dc.dc_code,
                ip.property_value as d_profile,
                ip2.property_value as actute_chronic
                FROM
                line_items
                LEFT JOIN variants ON line_items.variant_id = variants.id
                LEFT JOIN products ON variants.product_id = products.id
                LEFT JOIN product_types ON products.product_type_id = product_types.id
                LEFT JOIN categories ON categories.id = product_types.category_id
                LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
                LEFT JOIN orders o ON o.id = osl.order_id
                LEFT JOIN distribution_centers dc ON dc.id = o.store_id
                LEFT JOIN item_properties ip ON ip.item_id = line_items.variant_id AND ip.property_id IN ('1351')
                LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
                LEFT JOIN users on users.id = osl.modified_by_id
                WHERE
                line_items.order_id IN (
                    SELECT order_id
                    FROM order_status_logs as ols
                    LEFT JOIN orders ON orders.id = ols.order_id
                    WHERE
                        state_changed_on >= '2024-12-31 18:30:00.000000'
                        AND to_state = 3
                        AND city_id = 13
                )
                AND osl.to_state = 3
                ORDER BY confirmed_on DESC
                ''')
                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]  # Get column names from cursor description
                sales = pd.DataFrame(records, columns=columns)  # Convert records to DataFrame

                logging.info(f"Fetched {len(records)} records from the database.")

        except (Exception, psycopg2.Error) as error:
            logging.error(f"Error connecting to PostgreSQL: {error}")
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                logging.info("PostgreSQL connection is closed.")

        # Updating data into Google Sheet
        gsheet_name = 'CC Order Dashboard'
        tab_name = 'Sales'

        def write_df_to_gsheet(gsheet_name, tab_name, sales):
            try:
                gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
                sh = gc.open_by_key("1_ibELjBzTBBjKeNkgsIm5EzrDo5vamHZraOjMjDqNvI")  # Key of the Google Sheet
                worksheet = sh.worksheet(tab_name)
                set_with_dataframe(worksheet, sales)
                logging.info(f"Sales data successfully updated in {gsheet_name} Sheet.")
            except Exception as e:
                logging.error(f"Error updating Google Sheet: {e}")

        write_df_to_gsheet(gsheet_name, tab_name, sales)
    except Exception as e:
        logging.error(f"Unexpected error in job execution: {e}")
    finally:
        job_running.release()
        logging.info("Job finished.\n")

# Schedule the job to run every hour at minute 02
schedule.every().hour.at(":03").do(line_items_sales)
logging.info("Job scheduled to run every hour.")

# Main loop to keep the scheduler running
while True:
    schedule.run_pending()
    time.sleep(60)  # Check for scheduled jobs every 60 seconds


In [ ]:
# Sale update in CC_Dashboard- Added logging and threading : old
import schedule
import time
import threading
import pandas as pd
import logging
from sshtunnel import SSHTunnelForwarder
import psycopg2
import gspread
from gspread_dataframe import set_with_dataframe

# Configure logging
logging.basicConfig(
    filename="sales_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

job_running = threading.Lock()

def line_items_sales():
    if not job_running.acquire(blocking=False):
        logging.warning("Job is already running, skipping this execution.")
        return

    try:
        logging.info("Job started.")
        connection = None  # Initialize connection to None at the start
        cursor = None  # Initialize cursor as None as well

        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                    host='127.0.0.1',
                    port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                    database='efrprod',
                    user='emamireaduser',
                    password='emamireadaccess'
                )
                cursor = connection.cursor()

                # Fetching item-wise sales data
                cursor.execute('''
                SELECT
                line_items.order_id as ord_id,
                line_items.variant_id,
                products.name as product_name,
                categories.name as category,
                line_items.quantity,
                line_items.mrp_paise * 1.0 / 100 as MRP,
                line_items.sales_price_paise * 1.0 / 100 as Sales_price,
                line_items.total_paise * 1.0 / 100 as Total_Price,
                line_items.discount_amount_paise * 1.0 / 100 as Discount,
                --osl.modified_by_id,
                users.name as modified_by,
                CASE
                    WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
                    WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                    WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                    WHEN osl.channel = 0 THEN 'Call Center'
                    WHEN osl.channel = 1 THEN 'Saathi_App'
                    ELSE 'Website'
                END as channel,
                osl.state_changed_on + time '5:30' as confirmed_on,
                CASE
                    WHEN o.state = 0 THEN 'cart'
                    WHEN o.state = 1 THEN 'pre_checkout'
                    WHEN o.state = 2 THEN 'Checkout'
                    WHEN o.state = 6 THEN 'Delivered'
                    WHEN o.state = 7 THEN 'Cancelled'
                    WHEN o.state = 14 THEN 'Return_completed'
                    ELSE 'In-Progress'
                END as status,
                dc.dc_code,
                ip.property_value as d_profile,
                ip2.property_value as actute_chronic
                FROM
                line_items
                LEFT JOIN variants ON line_items.variant_id = variants.id
                LEFT JOIN products ON variants.product_id = products.id
                LEFT JOIN product_types ON products.product_type_id = product_types.id
                LEFT JOIN categories ON categories.id = product_types.category_id
                LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
                LEFT JOIN orders o ON o.id = osl.order_id
                LEFT JOIN distribution_centers dc ON dc.id = o.store_id
                LEFT JOIN item_properties ip ON ip.item_id = line_items.variant_id AND ip.property_id IN ('1351')
                LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
                LEFT JOIN users on users.id = osl.modified_by_id
                WHERE
                line_items.order_id IN (
                    SELECT order_id
                    FROM order_status_logs as ols
                    LEFT JOIN orders ON orders.id = ols.order_id
                    WHERE
                        state_changed_on >= '2024-12-31 18:30:00.000000'
                        AND to_state = 3
                        AND city_id = 13
                )
                AND osl.to_state = 3
                ORDER BY confirmed_on DESC
                ''')
                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]  # Get column names from cursor description
                sales = pd.DataFrame(records, columns=columns)  # Convert records to DataFrame

                logging.info(f"Fetched {len(records)} records from the database.")

        except (Exception, psycopg2.Error) as error:
            logging.error(f"Error connecting to PostgreSQL: {error}")
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                logging.info("PostgreSQL connection is closed.")

        # Updating data into Google Sheet
        gsheet_name = 'CC Order Dashboard'
        tab_name = 'Sales'

        def write_df_to_gsheet(gsheet_name, tab_name, sales):
            try:
                gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
                sh = gc.open_by_key("1_ibELjBzTBBjKeNkgsIm5EzrDo5vamHZraOjMjDqNvI")  # Key of the Google Sheet
                worksheet = sh.worksheet(tab_name)
                set_with_dataframe(worksheet, sales)
                logging.info("Sales data successfully updated in Google Sheet.")
            except Exception as e:
                logging.error(f"Error updating Google Sheet: {e}")

        write_df_to_gsheet(gsheet_name, tab_name, sales)
    except Exception as e:
        logging.error(f"Unexpected error in job execution: {e}")
    finally:
        job_running.release()
        logging.info("Job finished.\n")

# Schedule the job to run every hour at minute 02
schedule.every().hour.at(":02").do(line_items_sales)
logging.info("Job scheduled to run every hour at minute 02.")

# Main loop to keep the scheduler running
while True:
    schedule.run_pending()
    time.sleep(60)  # Check for scheduled jobs every 60 seconds


# Line Items Update Backup At : 14th Feb2025

In [ ]:
#Updated_at : 5th March to change data
#Added Threading, Defined Schedule function
import schedule
import time
import pandas as pd
import csv
import psycopg2
import paramiko
from sshtunnel import SSHTunnelForwarder
import gspread
import numpy as np
from gspread_dataframe import set_with_dataframe
import logging
import threading  # For running the scheduler in the background

# Configure logging
logging.basicConfig(
    filename="log_Line_items_update.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def line_items():
    connection = None  # Initialize connection to None at the start
    cursor = None      # Initialize cursor as None as well
    df_rosscare = pd.DataFrame()  # Initialize as empty DataFrame
    df_generic = pd.DataFrame()   # Initialize as empty DataFrame

    try:
        logging.info("Job started.")
        with SSHTunnelForwarder(
            ('65.1.183.184', 22),
            ssh_username='ubuntu',
            ssh_pkey="/home/rohit/emami-prod.pem",
            remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
            local_bind_address=('localhost', 5432)
        ) as tunnel:
            # Establish a connection to the RDS PostgreSQL database
            connection = psycopg2.connect(
                host='127.0.0.1',
                port=tunnel.local_bind_port,  # Local port bound to the SSH tunnel
                database='efrprod',
                user='emamireaduser',
                password='emamireadaccess'
            )
            cursor = connection.cursor()
            
            # Fetching Item Wise Sales Data for Rosscare
            logging.info("Fetching rosscare item-wise sales data")
            cursor.execute(
                """
                SELECT
                   line_items.order_id as ord_id,	
                   line_items.variant_id,
                   products.name as product_name,	
                   categories.name as category,
                   line_items.quantity,	
                   line_items.mrp_paise * 1.0 / 100 as MRP,	
                   line_items.sales_price_paise * 1.0 / 100 as Sales_price,	
                   line_items.total_paise * 1.0 / 100 as Total_Price,	
                   line_items.discount_amount_paise * 1.0 / 100 as Discount,
                   users.name as modified_by,
                   CASE 
                       WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
                       WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                       WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) THEN 'Ecom_CC'
                       WHEN osl.channel = 0 THEN 'Call Center'
                       WHEN osl.channel = 1 THEN 'Saathi_App'
                       ELSE 'Website'
                   END as channel,
                   osl.state_changed_on + Time '5:30' as confirmed_on,
                   CASE 
                       WHEN o.state = 0 THEN 'cart'
                       WHEN o.state = 1 THEN 'pre_checkout'
                       WHEN o.state = 2 THEN 'Checkout'
                       WHEN o.state = 6 THEN 'Delivered'
                       WHEN o.state = 7 THEN 'Cancelled'
                       WHEN o.state = 14 THEN 'Return_completed'
                       ELSE 'In-Progress' 
                   END as status,
                   dc.dc_code,
                   ip.property_value as d_profile,
                   ip2.property_value as actute_chronic
                FROM 
                   line_items 
                   LEFT JOIN variants ON line_items.variant_id = variants.id	
                   LEFT JOIN products ON variants.product_id = products.id	
                   LEFT JOIN product_types ON products.product_type_id = product_types.id	
                   LEFT JOIN categories ON categories.id = product_types.category_id
                   LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
                   LEFT JOIN orders o ON o.id = osl.order_id
                   LEFT JOIN distribution_centers dc ON dc.id = o.store_id
                   LEFT JOIN item_properties ip ON ip.item_id = line_items.variant_id AND ip.property_id IN ('1351')
                   LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
                   LEFT JOIN users ON users.id = osl.modified_by_id
                WHERE 
                   line_items.order_id IN (
                       SELECT order_id 
                       FROM order_status_logs as ols
                       LEFT JOIN orders ON orders.id = ols.order_id
                       WHERE 
                           state_changed_on >= '2024-10-31 18:30:00.000000' 
                           AND to_state = 3
                           AND city_id = 13
                   )
                   AND osl.to_state = 3
                   AND Variant_id in
                   (
                    35521,34399,36121,23363,23813,46601,52563,31312,42309,44329,33980,47540,30672,52651,52577,
                    35949,34764,36122,36131,44359,44539,52595,42307,42310,42315,30643,47549,30673,52641,52578,
                    35514,34765,36123,18276,34766,45505,31309,44368,44326,44330,30644,47550,23191,52646,52579,
                    35517,1460,36124,18277,34767,30585,23188,41250,42311,44371,30645,47551,30674,52611,52582,
                    35515,23808,23362,18278,36130,48976,23189,35518,42312,44331,30646,47552,30678,52653,52591,
                    35952,34757,36125,18279,34761,50084,23190,44332,42313,33979,30647,47558,30679,52648,52583,
                    43846,4342,34770,36132,34771,50592,30659,23270,42314,44370,43784,47559,30677,52639,52592,
                    44475,2918,33660,33918,43841,50816,30660,26881,44327,41249,30648,48356,30680,52612,52584,
                    17277,34450,25089,23802,34773,50823,18236,44372,44328,44152,30649,48357,35522,52633,52593,
                    17273,34096,31313,30111,44358,50822,34164,34160,35519,44153,30650,48358,35525,51436,52594,
                    17271,33788,21562,35524,44357,51682,34163,41256,35516,44154,44375,48363,30669,51437,52636,
                    28527,33779,36126,30273,43847,51686,42306,42326,37807,44155,44376,30656,36027,52652,52637,
                    29366,3798,23359,34772,44355,51687,30662,44325,37808,41255,44373,30657,52650,52640,52645,
                    28529,30253,18139,30274,44537,51683,30661,30664,25620,41253,45187,30658,50836,52649,52794,
                    28530,7929,36127,34758,44474,51684,30663,33982,26833,41252,45188,30665,51000,52205,52791,
                    28532,6718,36128,34769,47613,51685,35955,34154,30675,41254,44479,30768,50834,52287,52793,
                    28533,17282,36129,23812,47614,51562,41251,34155,30676,28366,54050,30666,47736,52570,52803,
                    17286,34763,38063,30298,47542,51561,28323,34165,44559,35513,52638,35523,47735,52571,52804,
                    17270,34776,26786,34774,47615,52063,33981,43753,44560,35520,51742,30667,51438,52572,52807,
                    3794,2930,23366,24337,43848,52064,44369,43752,44561,44374,45242,30668,51430,52574,52829,
                    23361,17276,23365,34760,45507,17269,33983,43751,34179,34162,47538,30670,51432,52575,52830,
                    36133,36120,23364,34759,45506,17275,42266,42308,35951,44558,47539,30671,52170,52576,52831,
                    52832,52833,52835,52836,52837,52838,52839,52840,52841,52842,52843,52844,52845,52846,52847,
                    52861,52864,52865,52866,52867,52868,52869,53995,53996,53997,53999,54000,53990,54001,54005,
                    54006,54007,53991,53989,54012,54013,54014,54052,54078,54079,53845,53884,53885,53998,53992,
                    52870,52871,52913,52914,53022,53109,53175,53190,53191,53192,53399,53688,53719,53843,53844,
                    53993,53994,54312,54423,54436,54438,54440,54441,54442,54443,54445
                   )
                ORDER BY osl.state_changed_on DESC
                """
            )
            records = cursor.fetchall()
            # Convert records to a DataFrame
            df_rosscare = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])
            df_rosscare['day'] = df_rosscare['confirmed_on'].dt.day
            df_rosscare['month_name'] = df_rosscare['confirmed_on'].dt.strftime('%b')
            logging.info(f"{len(records)} records fetched from Rosscare sales data")
            
            # Fetching Generic Sales Data
            logging.info("Fetching generic sales data")
            cursor.execute(
                """
                SELECT
                   line_items.order_id as ord_id,	
                   line_items.variant_id,
                   products.name as product_name,	
                   categories.name as category,
                   line_items.quantity,	
                   line_items.mrp_paise * 1.0 / 100 as MRP,	
                   line_items.sales_price_paise * 1.0 / 100 as Sales_price,	
                   line_items.total_paise * 1.0 / 100 as Total_Price,	
                   line_items.discount_amount_paise * 1.0 / 100 as Discount,
                   users.name as modified_by,
                   CASE 
                       WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
                       WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                       WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) THEN 'Ecom_CC'
                       WHEN osl.channel = 0 THEN 'Call Center'
                       WHEN osl.channel = 1 THEN 'Saathi_App'
                       ELSE 'Website'
                   END as channel,
                   osl.state_changed_on + Time '5:30' as confirmed_on,
                   CASE 
                       WHEN o.state = 0 THEN 'cart'
                       WHEN o.state = 1 THEN 'pre_checkout'
                       WHEN o.state = 2 THEN 'Checkout'
                       WHEN o.state = 6 THEN 'Delivered'
                       WHEN o.state = 7 THEN 'Cancelled'
                       WHEN o.state = 14 THEN 'Return_completed'
                       ELSE 'In-Progress' 
                   END as status,
                   dc.dc_code,
                   ip.property_value as d_profile,
                   ip2.property_value as actute_chronic
                FROM 
                   line_items 
                   LEFT JOIN variants ON line_items.variant_id = variants.id	
                   LEFT JOIN products ON variants.product_id = products.id	
                   LEFT JOIN product_types ON products.product_type_id = product_types.id	
                   LEFT JOIN categories ON categories.id = product_types.category_id
                   LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
                   LEFT JOIN orders o ON o.id = osl.order_id
                   LEFT JOIN distribution_centers dc ON dc.id = o.store_id
                   LEFT JOIN item_properties ip ON ip.item_id = line_items.variant_id AND ip.property_id IN ('1351')
                   LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
                   LEFT JOIN users ON users.id = osl.modified_by_id
                WHERE 
                   line_items.order_id IN (
                       SELECT order_id 
                       FROM order_status_logs as ols
                       LEFT JOIN orders ON orders.id = ols.order_id
                       WHERE 
                           state_changed_on >= '2024-10-31 18:30:00.000000' 
                           AND to_state = 3
                           AND city_id = 13
                   )
                   AND osl.to_state = 3
                   AND Variant_id in
                   (
                       21861,46497,51039,51748,52272,51925,52129,53336,53260,53697,21974,53939,54181,46466,53507,49021,53782,51787,
                       47267,46476,50852,47140,52273,51823,53409,53511,53253,53698,53922,53781,54182,53377,46719,53532,53500,53463,
                       46749,46692,52279,46831,52274,52262,52550,53215,53496,22656,53923,53266,54183,46511,52547,54140,47984,53492,
                       46488,22469,51226,46716,51781,51988,52537,53334,44619,53699,53924,45245,54184,50864,52538,53382,54144,53380,
                       47852,44922,36881,46743,51849,51854,53394,53339,46478,53700,48789,16684,53913,53408,53898,53383,53907,53340,
                       47164,46756,48139,46474,52259,52014,53374,53338,47780,53701,53925,53940,54185,52368,46869,53518,46968,53527,
                       47811,46826,22275,45653,52258,53690,53327,53337,30001,51378,53926,10070,22734,53379,53503,53514,53464,53345,
                       50888,46525,22276,46485,52260,52275,53329,2787,40104,53702,53927,17207,40709,53389,53822,22780,46914,52303,
                       22903,46526,22382,49546,52146,52276,53691,30256,53693,53703,53928,54169,45691,51780,46846,46492,53497,47800,
                       47532,47258,51747,46731,51945,52277,53513,30329,46167,53704,53227,50884,54186,50516,34257,47250,52544,46501,
                       50899,46535,50649,47097,52261,52278,53324,46694,47518,47702,21742,53508,49391,54146,54136,53352,46633,17110,
                       49892,51188,22902,46898,52181,51930,53325,53311,46753,46676,53929,46867,31300,53499,46734,53385,46823,53354,
                       46463,46899,50995,47341,52192,51881,53328,53265,17371,11516,53930,53454,52316,52533,54137,53378,53375,37290,
                       44720,46464,51965,36576,51828,51800,53189,53520,53502,43886,53931,48249,52246,53370,53900,54141,25759,53521,
                       50860,50855,50683,51069,52263,52282,51339,53515,42103,53705,53932,47088,52236,53501,53505,54142,53793,32967,
                       46536,51329,53689,46687,52264,52280,53692,5558,53694,53706,53933,47122,54187,53462,48142,40049,54145,53314,
                       46703,50928,40436,52267,52265,52281,47481,53316,53695,53919,53934,46748,54188,53355,53343,40050,53371,46523,
                       22415,51409,51744,52268,52180,50865,52099,53274,53696,53707,53935,54170,53546,50677,21935,54143,53387,46482,
                       47465,50937,51745,52269,52202,50667,53456,53273,22201,53708,53936,54171,46529,50724,54138,46951,53470,47760,
                       51746,22205,51715,52270,52266,51799,53335,53263,22202,53920,53937,50550,46917,53369,54139,53268,50847,53533,
                       22008,51138,46487,52271,52232,51852,48102,53262,46738,53921,53938,53482,52384,51037,40075,50915,53384,53512,
                       53906,54199,53441,53911,52294,53905,54200,54201,53388,53381,53903,54202,54203,53780,53902,54204,54205,46908,
                       54207,29979,49728,45243,17298,29588,53814,53784,54208,54209,54210,40363,54211,31299,54212,53473,53528,54213,
                       47206,53447,39960,54217,52178,54218,46534,34488,22246,46159,40501,47235,48188,22095,46807,46484,12636,54219,
                       54220,50499,48127,47083,52305,47624,49009,49207,40500,52251,40062,50076,54221,52315,46832,46837,47755,46531,
                       47052,54223,50491,22247,53264,52007,53530,54224,46889,54225,46669,50732,25544,53411,12352,46841,51008,54226,
                       52127,47872,34213,44536,40559,54228,46887,53258,22232,46480,54229,54230,49220,54231,47891,47100,54232,48247,
                       47767,54235,40061,46900,46521,53310,47149,50401,54236,22916,21523,54237,54238,54239,47591,47462,54240,52559,
                       54242,50185,50737,54243,46886,54244,47006,48640,46772,46465,53419,47240,52385,22151,54245,46727,53765,51242,
                       51012,54247,51856,38331,46891,54248,46518,47520,39956,54249,39837,54250,22414,54251,46726,54252,47273,46840,
                       22936,52553,46827,51146,47561,48467,50851,46502,47287,54163,53487,46517,46682,53396,46499,46507,22937,54168,
                       54147,17217,46677,47268,50977,43273,51052,54148,54156,54164,52186,47660,51814,54191,54151,46508,22471,50739,
                       52301,46520,52179,47876,54172,46721,54189,43530,47688,54165,40298,52365,47999,53272,54152,40361,54166,52546,
                       53766,53517,54158,40139,54173,46481,53498,46715,54157,46522,53912,46693,53516,54192,46892,54206,39669,47032,
                       47139,54153,54159,47506,54174,46754,54190,54149,47973,21884,39948,53522,51307,54193,54214,23224,54215,53519,
                       40499,53779,54160,46956,46822,46675,53890,22141,21864,44535,47272,54175,52239,54194,46751,48191,53386,52244,
                       53313,54154,54161,54167,37607,46746,53914,54150,47064,22234,40079,54176,52047,54195,47274,54222,22804,52310,
                       21873,54155,54162,22782,47582,45633,53445,21347,46472,22236,39965,54177,53819,54196,54246,46765,22012,37511,
                       47042,53220,48195,22783,46683,53524,53908,47603,22901,46515,48217,54178,51119,22132,52256,47432,46763,47484,
                       54179,46543,46856,52293,47601,46717,54227,22946,50890,54197,21875,21883,47777,53483,46777,53904,7453,50350,
                       53910,48145,54233,49562,48332,54234,47499,48672,50845,50907,54216,40703,54253,47602,53915,51376,46630,22569,
                       54180,46834,47752,46568,48670,54241,47085,46572,50853,54198,53766,47139,40499,53313,21873,47042,54179,53910,
                       44720,46488,49892,47852,46536,46703,46463,47811,47532,22415,47267,46749,47465,51746,22903,22936,54147,52301,
                       54180,53906,50888,50899,54207,47206,54220,47052,52127,47767,54242,51012,50860,22008,47164,54324,54325,54326,
                       54327,54328,54329,54330,54331,54332,48974,54333,46912,54334,54335,54336,54337,54338,52188,54339,46475,48605,
                       47106,40253,54340,53562,54341,53878,53233,46875,53234,54342,54343,54344,51180,54345,52311,47145,54346,54347,
                       50714,52420,54348,54349,54350,54351,54352,54353,54354,22133,54355,54356,54357,34219,50908,50886,46052,54358,
                       54359,54360,54361,54362,54363,54364,54365,54366,54367,54368,52383,54369,54370,54371,50711,52378,53322,53326,
                       47129,52409,47241,48028,54372,54373,54374,54375,54376,53315,22094,50862,52292,54377,46865,54378,54379,49447,
                       54380,54381,54382,45932,54383,54384,46708,54385,54386,22378,54387,22175,54388,54389,54390,54391,46933,54392,
                       53261,54393,54394,47306,54395,54396,54397,54398,54399,54400,54401,54402,54403,54404,54405,54406,54407,54408,
                       54409,54410,54411,54412,54413,50974,54414,54415,54416,54417,54418
                   )
                ORDER BY osl.state_changed_on DESC
                """
            )
            records = cursor.fetchall()
            # Convert records to a DataFrame
            df_generic = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])
            df_generic['day'] = df_generic['confirmed_on'].dt.day
            df_generic['month_name'] = df_generic['confirmed_on'].dt.strftime('%b')
            logging.info(f"{len(records)} records fetched from generic sales data")
            
    except (Exception, psycopg2.Error) as error:
        logging.error(f"Error connecting to PostgreSQL: {error}")
    finally:
        # Close the database connection if it was opened
        if connection:
            cursor.close()
            connection.close()
            logging.info("PostgreSQL connection is closed")
    
    # Function to update a given DataFrame to a specified Google Sheets tab
    def write_df_to_gsheet(gsheet_name, tab_name, df):
        gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
        sh = gc.open_by_key("1FRgKGBPXjg1mjncLsohsKmc9DGIqQZyDZSx6T5qy1ds")
        worksheet = sh.worksheet(tab_name)
        set_with_dataframe(worksheet, df)
        

    # Update Rosscare data
    gsheet_name = 'Automation data'
    tab_name = 'Rosscare'
    logging.info(f"Updating rosscare sales data in {gsheet_name}")
    write_df_to_gsheet(gsheet_name, tab_name, df_rosscare)
    logging.info("Rosscare data updated")
    # Update Generic data
    tab_name = 'Generic'
    logging.info(f"Updating generic sales data in {gsheet_name}")
    write_df_to_gsheet(gsheet_name, tab_name, df_generic)
    logging.info("Generic data updated\n")

# Schedule the job to run every hour at the 31st minute
schedule.every().hour.at(":50").do(line_items)
logging.info("Scheduling task to run every hour")

# Define a function to run the scheduler in a background thread
def run_scheduler():
    while True:
        schedule.run_pending()
        time.sleep(5)  # Check every 5 seconds

# Start the scheduler thread (as a daemon so it stops when you close the Notebook)
scheduler_thread = threading.Thread(target=run_scheduler, daemon=True)
scheduler_thread.start()

logging.info("Scheduler is running in the background. Check your log file for updates.")


In [ ]:
#Updated at : 14th feb 2025 -logging configured
import schedule
import time
import pandas as pd
import csv
import psycopg2
import paramiko
from sshtunnel import SSHTunnelForwarder
import gspread
import numpy as np
from gspread_dataframe import set_with_dataframe
import logging

logging.basicConfig(
    filename = "log_Line_items_update.txt",
    level = logging.INFO,
    format = "%(asctime)s - %(levelname)s - %(message)s"
)

def line_items():
    connection = None  # Initialize connection to None at the start
    cursor = None  # Initialize cursor as None as well

    try:
        logging.info("Job started.")
        with SSHTunnelForwarder(
            ('65.1.183.184', 22),
            ssh_username='ubuntu',
            ssh_pkey="/home/rohit/emami-prod.pem",
            remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
            local_bind_address=('localhost', 5432)
        ) as tunnel:
            # Establish a connection to the RDS PostgreSQL database
            connection = psycopg2.connect(
                host='127.0.0.1',
                port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                database='efrprod',
                user='emamireaduser',
                password='emamireadaccess'
            )
            cursor = connection.cursor()
             #Fetching Item Wise Sales Data all categories
            logging.info("Fetching rosscare item-wise sales data")
            cursor.execute(
           """
            SELECT
           line_items.order_id as ord_id,
           line_items.variant_id,
           products.name as product_name,
           categories.name as category,
           line_items.quantity,
           line_items.mrp_paise * 1.0 / 100 as MRP,
           line_items.sales_price_paise * 1.0 / 100 as Sales_price,
           line_items.total_paise * 1.0 / 100 as Total_Price,
           line_items.discount_amount_paise * 1.0 / 100 as Discount,
           --osl.modified_by_id,
           users.name as modified_by,
           CASE
               WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
               WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
               WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
               WHEN osl.channel = 0 THEN 'Call Center'
               WHEN osl.channel = 1 THEN 'Saathi_App'
               ELSE 'Website'
           END as channel,
           osl.state_changed_on + Time '5:30' as confirmed_on,
           CASE
               WHEN o.state = 0 THEN 'cart'
               WHEN o.state = 1 THEN 'pre_checkout'
               WHEN o.state = 2 THEN 'Checkout'
               WHEN o.state = 6 THEN 'Delivered'
               WHEN o.state = 7 THEN 'Cancelled'
               WHEN o.state = 14 THEN 'Return_completed'
               ELSE 'In-Progress'
           END as status,
           dc.dc_code,
           ip.property_value as d_profile,
           ip2.property_value as actute_chronic
           FROM
           line_items
           LEFT JOIN variants ON line_items.variant_id = variants.id
           LEFT JOIN products ON variants.product_id = products.id
           LEFT JOIN product_types ON products.product_type_id = product_types.id
           LEFT JOIN categories ON categories.id = product_types.category_id
           LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
           LEFT JOIN orders o ON o.id = osl.order_id
           LEFT JOIN distribution_centers dc ON dc.id = o.store_id
           LEFT JOIN item_properties ip ON ip.item_id = line_items.variant_id AND ip.property_id IN ('1351')
           LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
           left JOIn users on users.id = osl.modified_by_id
           WHERE
           line_items.order_id IN (
               SELECT order_id
               FROM order_status_logs as ols
               LEFT JOIN orders ON orders.id = ols.order_id
               WHERE
                   state_changed_on >= '2024-09-30 18:30:00.000000'
                   AND to_state = 3
                   AND city_id = 13
           )
           AND osl.to_state = 3
           AND Variant_id in
           (
            35521	,34399	,36121	,23363	,23813	,46601	,52563	,31312	,42309	,44329	,33980	,47540	,30672	,52651	,52577
            ,35949	,34764	,36122	,36131	,44359	,44539	,52595	,42307	,42310	,42315	,30643	,47549	,30673	,52641	,52578
            ,35514	,34765	,36123	,18276	,34766	,45505	,31309	,44368	,44326	,44330	,30644	,47550	,23191	,52646	,52579
            ,35517	,1460	,36124	,18277	,34767	,30585	,23188	,41250	,42311	,44371	,30645	,47551	,30674	,52611	,52582
            ,35515	,23808	,23362	,18278	,36130	,48976	,23189	,35518	,42312	,44331	,30646	,47552	,30678	,52653	,52591
            ,35952	,34757	,36125	,18279	,34761	,50084	,23190	,44332	,42313	,33979	,30647	,47558	,30679	,52648	,52583
            ,43846	,4342	,34770	,36132	,34771	,50592	,30659	,23270	,42314	,44370	,43784	,47559	,30677	,52639	,52592
            ,44475	,2918	,33660	,33918	,43841	,50816	,30660	,26881	,44327	,41249	,30648	,48356	,30680	,52612	,52584
            ,17277	,34450	,25089	,23802	,34773	,50823	,18236	,44372	,44328	,44152	,30649	,48357	,35522	,52633	,52593
            ,17273	,34096	,31313	,30111	,44358	,50822	,34164	,34160	,35519	,44153	,30650	,48358	,35525	,51436	,52594
            ,17271	,33788	,21562	,35524	,44357	,51682	,34163	,41256	,35516	,44154	,44375	,48363	,30669	,51437	,52636
            ,28527	,33779	,36126	,30273	,43847	,51686	,42306	,42326	,37807	,44155	,44376	,30656	,36027	,52652	,52637
            ,29366	,3798	,23359	,34772	,44355	,51687	,30662	,44325	,37808	,41255	,44373	,30657	,52650	,52640	,52645
            ,28529	,30253	,18139	,30274	,44537	,51683	,30661	,30664	,25620	,41253	,45187	,30658	,50836	,52649	,52794
            ,28530	,7929	,36127	,34758	,44474	,51684	,30663	,33982	,26833	,41252	,45188	,30665	,51000	,52205	,52791
            ,28532	,6718	,36128	,34769	,47613	,51685	,35955	,34154	,30675	,41254	,44479	,30768	,50834	,52287	,52793
            ,28533	,17282	,36129	,23812	,47614	,51562	,41251	,34155	,30676	,28366	,54050	,30666	,47736	,52570	,52803
            ,17286	,34763	,38063	,30298	,47542	,51561	,28323	,34165	,44559	,35513	,52638	,35523	,47735	,52571	,52804
            ,17270	,34776	,26786	,34774	,47615	,52063	,33981	,43753	,44560	,35520	,51742	,30667	,51438	,52572	,52807
            ,3794	,2930	,23366	,24337	,43848	,52064	,44369	,43752	,44561	,44374	,45242	,30668	,51430	,52574	,52829
            ,23361	,17276	,23365	,34760	,45507	,17269	,33983	,43751	,34179	,34162	,47538	,30670	,51432	,52575	,52830
            ,36133	,36120	,23364	,34759	,45506	,17275	,42266	,42308	,35951	,44558	,47539	,30671	,52170	,52576	,52831
            ,52832	,52833	,52835	,52836	,52837	,52838	,52839	,52840	,52841	,52842	,52843	,52844	,52845	,52846	,52847
            ,52861	,52864	,52865	,52866	,52867	,52868	,52869	,53995	,53996	,53997	,53999	,54000	,53990	,54001	,54005
            ,54006	,54007	,53991	,53989	,54012	,54013	,54014	,54052	,54078	,54079	,53845	,53884	,53885	,53998	,53992
            ,52870	,52871	,52913	,52914	,53022	,53109	,53175	,53190	,53191	,53192	,53399	,53688	,53719	,53843	,53844
            ,53993	,53994	,54312	,54423	,54436	,54438	,54440	,54441	,54442	,54443	,54445				)

           order by osl.state_changed_on desc
           """

           )
            records = cursor.fetchall()
            # Convert to DataFrame instead of writing to a CSV file
            df_rosscare = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])
            df_rosscare['day'] = df_rosscare['confirmed_on'].dt.day
            df_rosscare['month_name'] = df_rosscare['confirmed_on'].dt.strftime('%b')
            logging.info(f"{len(records)} fetched from sales data")
            logging.info("Fetching generic sales data")
            cursor.execute(
           """
            SELECT
           line_items.order_id as ord_id,
           line_items.variant_id,
           products.name as product_name,
           categories.name as category,
           line_items.quantity,
           line_items.mrp_paise * 1.0 / 100 as MRP,
           line_items.sales_price_paise * 1.0 / 100 as Sales_price,
           line_items.total_paise * 1.0 / 100 as Total_Price,
           line_items.discount_amount_paise * 1.0 / 100 as Discount,
           --osl.modified_by_id,
           users.name as modified_by,
           CASE
               WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
               WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
               WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
               WHEN osl.channel = 0 THEN 'Call Center'
               WHEN osl.channel = 1 THEN 'Saathi_App'
               ELSE 'Website'
           END as channel,
           osl.state_changed_on + Time '5:30' as confirmed_on,
           CASE
               WHEN o.state = 0 THEN 'cart'
               WHEN o.state = 1 THEN 'pre_checkout'
               WHEN o.state = 2 THEN 'Checkout'
               WHEN o.state = 6 THEN 'Delivered'
               WHEN o.state = 7 THEN 'Cancelled'
               WHEN o.state = 14 THEN 'Return_completed'
               ELSE 'In-Progress'
           END as status,
           dc.dc_code,
           ip.property_value as d_profile,
           ip2.property_value as actute_chronic
           FROM
           line_items
           LEFT JOIN variants ON line_items.variant_id = variants.id
           LEFT JOIN products ON variants.product_id = products.id
           LEFT JOIN product_types ON products.product_type_id = product_types.id
           LEFT JOIN categories ON categories.id = product_types.category_id
           LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
           LEFT JOIN orders o ON o.id = osl.order_id
           LEFT JOIN distribution_centers dc ON dc.id = o.store_id
           LEFT JOIN item_properties ip ON ip.item_id = line_items.variant_id AND ip.property_id IN ('1351')
           LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
           left JOIn users on users.id = osl.modified_by_id
           WHERE
           line_items.order_id IN (
               SELECT order_id
               FROM order_status_logs as ols
               LEFT JOIN orders ON orders.id = ols.order_id
               WHERE
                   state_changed_on >= '2024-09-30 18:30:00.000000'
                   AND to_state = 3
                   AND city_id = 13
           )
           AND osl.to_state = 3
           AND Variant_id in
           (
               21861,46497	,51039	,51748	,52272	,51925	,52129	,53336	,53260	,53697	,21974	,53939	,54181	,46466	,53507	,49021	,53782	,51787
            ,47267	,46476	,50852	,47140	,52273	,51823	,53409	,53511	,53253	,53698	,53922	,53781	,54182	,53377	,46719	,53532	,53500	,53463
            ,46749	,46692	,52279	,46831	,52274	,52262	,52550	,53215	,53496	,22656	,53923	,53266	,54183	,46511	,52547	,54140	,47984	,53492
            ,46488	,22469	,51226	,46716	,51781	,51988	,52537	,53334	,44619	,53699	,53924	,45245	,54184	,50864	,52538	,53382	,54144	,53380
            ,47852	,44922	,36881	,46743	,51849	,51854	,53394	,53339	,46478	,53700	,48789	,16684	,53913	,53408	,53898	,53383	,53907	,53340
            ,47164	,46756	,48139	,46474	,52259	,52014	,53374	,53338	,47780	,53701	,53925	,53940	,54185	,52368	,46869	,53518	,46968	,53527
            ,47811	,46826	,22275	,45653	,52258	,53690	,53327	,53337	,30001	,51378	,53926	,10070	,22734	,53379	,53503	,53514	,53464	,53345
            ,50888	,46525	,22276	,46485	,52260	,52275	,53329	,2787	,40104	,53702	,53927	,17207	,40709	,53389	,53822	,22780	,46914	,52303
            ,22903	,46526	,22382	,49546	,52146	,52276	,53691	,30256	,53693	,53703	,53928	,54169	,45691	,51780	,46846	,46492	,53497	,47800
            ,47532	,47258	,51747	,46731	,51945	,52277	,53513	,30329	,46167	,53704	,53227	,50884	,54186	,50516	,34257	,47250	,52544	,46501
            ,50899	,46535	,50649	,47097	,52261	,52278	,53324	,46694	,47518	,47702	,21742	,53508	,49391	,54146	,54136	,53352	,46633	,17110
            ,49892	,51188	,22902	,46898	,52181	,51930	,53325	,53311	,46753	,46676	,53929	,46867	,31300	,53499	,46734	,53385	,46823	,53354
            ,46463	,46899	,50995	,47341	,52192	,51881	,53328	,53265	,17371	,11516	,53930	,53454	,52316	,52533	,54137	,53378	,53375	,37290
            ,44720	,46464	,51965	,36576	,51828	,51800	,53189	,53520	,53502	,43886	,53931	,48249	,52246	,53370	,53900	,54141	,25759	,53521
            ,50860	,50855	,50683	,51069	,52263	,52282	,51339	,53515	,42103	,53705	,53932	,47088	,52236	,53501	,53505	,54142	,53793	,32967
            ,46536	,51329	,53689	,46687	,52264	,52280	,53692	,5558	,53694	,53706	,53933	,47122	,54187	,53462	,48142	,40049	,54145	,53314
            ,46703	,50928	,40436	,52267	,52265	,52281	,47481	,53316	,53695	,53919	,53934	,46748	,54188	,53355	,53343	,40050	,53371	,46523
            ,22415	,51409	,51744	,52268	,52180	,50865	,52099	,53274	,53696	,53707	,53935	,54170	,53546	,50677	,21935	,54143	,53387	,46482
            ,47465	,50937	,51745	,52269	,52202	,50667	,53456	,53273	,22201	,53708	,53936	,54171	,46529	,50724	,54138	,46951	,53470	,47760
            ,51746	,22205	,51715	,52270	,52266	,51799	,53335	,53263	,22202	,53920	,53937	,50550	,46917	,53369	,54139	,53268	,50847	,53533
            ,22008	,51138	,46487	,52271	,52232	,51852	,48102	,53262	,46738	,53921	,53938	,53482	,52384	,51037	,40075	,50915	,53384	,53512
            ,53906	,54199	,53441	,53911	,52294	,53905	,54200	,54201	,53388	,53381	,53903	,54202	,54203	,53780	,53902	,54204	,54205	,46908
            ,54207	,29979	,49728	,45243	,17298	,29588	,53814	,53784	,54208	,54209	,54210	,40363	,54211	,31299	,54212	,53473	,53528	,54213
            ,47206	,53447	,39960	,54217	,52178	,54218	,46534	,34488	,22246	,46159	,40501	,47235	,48188	,22095	,46807	,46484	,12636	,54219
            ,54220	,50499	,48127	,47083	,52305	,47624	,49009	,49207	,40500	,52251	,40062	,50076	,54221	,52315	,46832	,46837	,47755	,46531
            ,47052	,54223	,50491	,22247	,53264	,52007	,53530	,54224	,46889	,54225	,46669	,50732	,25544	,53411	,12352	,46841	,51008	,54226
            ,52127	,47872	,34213	,44536	,40559	,54228	,46887	,53258	,22232	,46480	,54229	,54230	,49220	,54231	,47891	,47100	,54232	,48247
            ,47767	,54235	,40061	,46900	,46521	,53310	,47149	,50401	,54236	,22916	,21523	,54237	,54238	,54239	,47591	,47462	,54240	,52559
            ,54242	,50185	,50737	,54243	,46886	,54244	,47006	,48640	,46772	,46465	,53419	,47240	,52385	,22151	,54245	,46727	,53765	,51242
            ,51012	,54247	,51856	,38331	,46891	,54248	,46518	,47520	,39956	,54249	,39837	,54250	,22414	,54251	,46726	,54252	,47273	,46840
            ,22936	,52553	,46827	,51146	,47561	,48467	,50851	,46502	,47287	,54163	,53487	,46517	,46682	,53396	,46499	,46507	,22937	,54168
            ,54147	,17217	,46677	,47268	,50977	,43273	,51052	,54148	,54156	,54164	,52186	,47660	,51814	,54191	,54151	,46508	,22471	,50739
            ,52301	,46520	,52179	,47876	,54172	,46721	,54189	,43530	,47688	,54165	,40298	,52365	,47999	,53272	,54152	,40361	,54166	,52546
            ,53766	,53517	,54158	,40139	,54173	,46481	,53498	,46715	,54157	,46522	,53912	,46693	,53516	,54192	,46892	,54206	,39669	,47032
            ,47139	,54153	,54159	,47506	,54174	,46754	,54190	,54149	,47973	,21884	,39948	,53522	,51307	,54193	,54214	,23224	,54215	,53519
            ,40499	,53779	,54160	,46956	,46822	,46675	,53890	,22141	,21864	,44535	,47272	,54175	,52239	,54194	,46751	,48191	,53386	,52244
            ,53313	,54154	,54161	,54167	,37607	,46746	,53914	,54150	,47064	,22234	,40079	,54176	,52047	,54195	,47274	,54222	,22804	,52310
            ,21873	,54155	,54162	,22782	,47582	,45633	,53445	,21347	,46472	,22236	,39965	,54177	,53819	,54196	,54246	,46765	,22012	,37511
            ,47042	,53220	,48195	,22783	,46683	,53524	,53908	,47603	,22901	,46515	,48217	,54178	,51119	,22132	,52256	,47432	,46763	,47484
            ,54179	,46543	,46856	,52293	,47601	,46717	,54227	,22946	,50890	,54197	,21875	,21883	,47777	,53483	,46777	,53904	,7453	,50350
            ,53910	,48145	,54233	,49562	,48332	,54234	,47499	,48672	,50845	,50907	,54216	,40703	,54253	,47602	,53915	,51376	,46630	,22569
            ,54180	,46834	,47752	,46568	,48670	,54241	,47085	,46572	,50853	,54198	,53766	,47139	,40499	,53313	,21873	,47042	,54179	,53910
            ,44720	,46488	,49892	,47852	,46536	,46703	,46463	,47811	,47532	,22415	,47267	,46749	,47465	,51746	,22903	,22936	,54147	,52301
            ,54180	,53906	,50888	,50899	,54207	,47206	,54220	,47052	,52127	,47767	,54242	,51012	,50860	,22008	,47164	,54324	,54325	,54326
            ,54327	,54328	,54329	,54330	,54331	,54332	,48974	,54333	,46912	,54334	,54335	,54336	,54337	,54338	,52188	,54339	,46475	,48605
            ,47106	,40253	,54340	,53562	,54341	,53878	,53233	,46875	,53234	,54342	,54343	,54344	,51180	,54345	,52311	,47145	,54346	,54347
            ,50714	,52420	,54348	,54349	,54350	,54351	,54352	,54353	,54354	,22133	,54355	,54356	,54357	,34219	,50908	,50886	,46052	,54358
            ,54359	,54360	,54361	,54362	,54363	,54364	,54365	,54366	,54367	,54368	,52383	,54369	,54370	,54371	,50711	,52378	,53322	,53326
            ,47129	,52409	,47241	,48028	,54372	,54373	,54374	,54375	,54376	,53315	,22094	,50862	,52292	,54377	,46865	,54378	,54379	,49447
            ,54380	,54381	,54382	,45932	,54383	,54384	,46708	,54385	,54386	,22378	,54387	,22175	,54388	,54389	,54390	,54391	,46933	,54392
            ,53261	,54393	,54394	,47306	,54395	,54396	,54397	,54398	,54399	,54400	,54401	,54402	,54403	,54404	,54405	,54406	,54407	,54408
            ,54409	,54410	,54411	,54412	,54413	,50974	,54414	,54415	,54416	,54417	,54418
           )
           order by osl.state_changed_on desc
           """

           )

            records = cursor.fetchall()
            # Convert to DataFrame instead of writing to a CSV file
            df_generic = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])
            df_generic['day'] = df_generic['confirmed_on'].dt.day
            df_generic['month_name'] = df_generic['confirmed_on'].dt.strftime('%b')
            logging.info(f"{len(records)} fetched from sales data")
    except (Exception, psycopg2.Error) as error:
        logging.info(f"Error connecting to PostgreSQL: {error}")
    finally:
        # Close the database connection
        if connection:
            cursor.close()
            connection.close()
            logging.info(f"PostgreSQL connection is closed")

    # Updating Data into Automation Sheet Rosscare Tab
    gsheet_name = 'Automation data' #This google sheet will be updated
    tab_name = 'Rosscare' # This particular tab is to be updated
    def write_df_to_gsheet (gsheet_name,tab_name,df_rosscare):
        gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
        sh = gc.open_by_key("1FRgKGBPXjg1mjncLsohsKmc9DGIqQZyDZSx6T5qy1ds") #Key Of the google sheet - Automation data
        worksheet = sh.worksheet(tab_name)
        set_with_dataframe(worksheet,df_rosscare)
        logging.info(f"{tab_name} Updated")
    logging.info(f"updating rosscare sales data in {gsheet_name}")
    write_df_to_gsheet(gsheet_name,tab_name,df_rosscare)
    print("Rosscare Updated_at", pd.Timestamp.now())

    # Updating Data into Automation Sheet Generic Tab
    gsheet_name = 'Automation data' #This google sheet will be updated
    tab_name = 'Generic' # This particular tab is to be updated
    def write_df_to_gsheet (gsheet_name,tab_name,df_generic):
        gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
        sh = gc.open_by_key("1FRgKGBPXjg1mjncLsohsKmc9DGIqQZyDZSx6T5qy1ds") #Key Of the google sheet - Automation data
        worksheet = sh.worksheet(tab_name)
        set_with_dataframe(worksheet,df_generic)
        logging.info(f"{tab_name} Updated\n")
    logging.info(f"updating generic sales data in {gsheet_name}")
    write_df_to_gsheet(gsheet_name,tab_name,df_generic)



schedule.every().hour.at(":04").do(line_items)
logging.info("Scheduling Task to run Everuy hour")
while True:
    schedule.run_pending()
    time.sleep(5)

# Bounce Report

In [ ]:
import schedule as schedule
import time
def bounce_order_report():
    try:
        import csv
        import pandas as pd
        import numpy as np
        import psycopg2
        import pandas as pd
        import paramiko
        import re
        import schedule
        import time
        from sshtunnel import SSHTunnelForwarder
        import gspread
        from gspread_dataframe import set_with_dataframe
        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                        database='efrprod',
                        user='emamireaduser',
                        password='emamireadaccess'
                )
                cursor = connection.cursor()

                # Perform database operations here
                cursor.execute(
        '''

        SELECT
        Bounce_tbl.order_id,
        Bounce_tbl.status,
        Bounce_tbl.sku,
        Bounce_tbl.name,
        ord_qty.order_qty,
        Bounce_tbl.deficit_quantity,
        Bounce_tbl.Level_1_store,
        Bounce_tbl.pincode,
        Bounce_tbl.confirmed_on
        FROM
        (with cte_bounce_orders as (
        SELECT
            o.order_id,
            CASE
        		WHEN orders.state =0 THEN 'cart'
        		WHEN orders.state =1 THEN 'pre_checkout'
        		WHEN orders.state =2 THEN 'Checkout'
        		WHEN orders.state =6 THEN 'Delivered'
        		WHEN orders.state =7 THEN 'Cancelled'
        		WHEN orders.state =14  THEN 'Return_completed'
        		ELSE 'In-Progress'
        		END AS status,
            v.sku,
            v.name,
            o.deficit_quantity,
            w.store_name AS Level_1_store,
            ar.pincode,
            ols.state_changed_on + INTERVAL '5 hours 30 minutes' AS confirmed_on,
        	CONCAT (o.order_id,v.sku) as order_fr_code,
            CASE
        	WHEN orders.customer_id IS NULL AND ols.channel = 1 THEN 'Mobile'
        	WHEN ols.channel = 0 AND COALESCE(orders.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
        	WHEN ols.channel = 0 THEN 'Call Center'
        	WHEN ols.channel = 1 THEN 'Saathi App'
        	ELSE 'Website'
        END as channel
        FROM
            order_deficit_quantity_logs o
        LEFT JOIN
            warehouses w ON o.warehouse_id = w.id
        LEFT JOIN
            areas ar ON o.area_id = ar.id
        LEFT JOIN
            variants v ON v.id = o.variant_id
        LEFT JOIN
            orders ON orders.id = o.order_id
        LEFT JOIN
            order_status_logs ols ON ols.order_id = o.order_id
        WHERE
            ols.to_state = 3
            AND ols.state_changed_on > '2024-11-30 18:30:00'
            AND orders.state > 2
            AND ols.state_changed_on + INTERVAL '5 hours 30 minutes' > '2024-12-31 18:30:00'
        )
        select * From cte_bounce_orders where confirmed_on > '2024-12-31 18:30:00') Bounce_tbl

        left join

        (
        SELECT
        CONCAT (order_id,'FR-',variant_id) as order_fr_code,
        quantity as order_qty
        FROM line_items where order_id in
        (
        SELECT
        order_id
        FROM order_status_logs
        WHERE state_changed_on >'2024-12-31 18:30' AND to_state = 3
        )) ord_qty
        on ord_qty.order_fr_code = Bounce_tbl.order_fr_code

        ''')
                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]  # Get column names from cursor description
                bounced_orders = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
                bounced_orders.to_csv('bounced_orders.csv')
                print("Data fetched and saved to bounced_orders.csv")
        except (Exception, psycopg2.Error) as error:
            print("Error connecting to PostgreSQL:", error)
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                print("PostgreSQL connection is closed")


        import smtplib
        from email.mime.multipart import MIMEMultipart
        from email.mime.text import MIMEText
        from email.mime.base import MIMEBase
        from email import encoders

        def send_email(sender_email, sender_password, recipient_emails, cc_emails, subject, message, attachment_filename):
            # Construct the email body with signature
            body_with_signature = f"{message}\n\nThanks & Regards,\nKumar Rohit\nAnalytics Manager | Emami Frank Ross Limited\n8882041300\nwww.frankrossphramacy.com"

            # Set up the MIME
            msg = MIMEMultipart()
            msg['From'] = sender_email
            msg['To'] = ', '.join(recipient_emails)
            msg['Cc'] = ', '.join(cc_emails) if cc_emails else None
            msg['Subject'] = subject

            # Add message body
            msg.attach(MIMEText(body_with_signature, 'plain'))

            # Attach file
            with open(attachment_filename, "rb") as attachment:
                part = MIMEBase("application", "octet-stream")
                part.set_payload(attachment.read())

            encoders.encode_base64(part)
            part.add_header(
                "Content-Disposition",
                f"attachment; filename= {attachment_filename}",
            )
            msg.attach(part)

            # Connect to the SMTP server
            with smtplib.SMTP('smtp-mail.outlook.com', 587) as server:
                server.starttls()  # Secure the connection
                server.login(sender_email, sender_password)  # Login to your email account
                # Combine the recipients and send the email
                recipients = recipient_emails + (cc_emails if cc_emails else [])
                server.sendmail(sender_email, recipients, msg.as_string())  # Send the email
                print("Email sent successfully")

        # Set your email credentials and email details
        sender_email = 'kumar.rohit@frankrosspharmacy.com'
        sender_password = 'Outl00k@0125'
        recipient_emails = ['debasish.das@frankrosspharmacy.com']  # List of primary recipients
        cc_emails = ['saurabh@frankrosspharmacy.com',
        'nitin.vyas@frankrosspharmacy.com','avijit.b@frankrosspharmacy.com','debopam.dey@frankrosspharmacy.com',
        'chiradeep.roygupta@frankrosspharmacy.com', 'saurabh@frankrosspharmacy.com','ajatia@frankrosspharmacy.com',
        'debtanu.baidya@frankrosspharmacy.com','sarit@frankrosspharmacy.com']
        subject = 'Bounced Article List'
        message = 'Hi Debasish,\n\nPFA excel file containing list of Bounced articles MTD'
        attachment_filename = 'bounced_orders.csv'  # Filename of the CSV file

        # Call the function to send the email with attachment
        send_email(sender_email, sender_password, recipient_emails, cc_emails, subject, message, attachment_filename)
        print("Updated at", pd.Timestamp.now())
    except (Exception, psycopg2.Error) as error:
        print("Error connecting to PostgreSQL:", error)
    finally:# Close the database connection
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

# Schedule the task to run every minute
schedule.every().day.at("10:15").do(bounce_order_report)
#print(schedule.get_jobs())
while True:
    schedule.run_pending()
    time.sleep(5)

# Cart Abondoned Users :                 Updated at : 11th Feb'2025

In [ ]:
# Configured Logging on 11th Feb and executed

import schedule
import time
import psycopg2
import pandas as pd
import paramiko
from sshtunnel import SSHTunnelForwarder
import gspread
from gspread_dataframe import set_with_dataframe
import csv
import numpy as np
import re
import os
import logging

# Configure logging
logging.basicConfig(
    filename="log_cart_abondoned.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

def cart_abondoned_orders():
    connection = None
    cursor = None

    try:
        logging.info("🚀 Starting cart_abondoned_orders script...")

        logging.info("🔗 Establishing SSH Tunnel...")
        with SSHTunnelForwarder(
            ('65.1.183.184', 22),
            ssh_username='ubuntu',
            ssh_pkey="/home/rohit/emami-prod.pem",
            remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
            local_bind_address=('localhost', 5432)
        ) as tunnel:
            logging.info("✅ SSH Tunnel established successfully.")

            # Establish PostgreSQL connection
            connection = psycopg2.connect(
                host='127.0.0.1',
                port=tunnel.local_bind_port,
                database='efrprod',
                user='emamireaduser',
                password='emamireadaccess'
            )
            cursor = connection.cursor()
            logging.info("✅ Database connection established.")

            # Execute SQL Query
            cursor.execute('''
            SELECT
                o.id as order_id,
                o.created_at + time '5:30' as cart_created_at,
                o.order_total_paise/100*1.0 as cart_value,
                u.name,
                ph.number,
                ph.otp_verified_at + time '5:30' as cust_registered_date,
                EXTRACT(DAY FROM o.created_at) as day
            FROM public.orders as o
            LEFT JOIN public.order_status_logs as osl
                ON osl.order_id = o.id
            LEFT JOIN users as u
                ON u.id = o.user_id
            LEFT JOIN phones as ph
                ON ph.user_id = o.user_id
            WHERE
                EXTRACT(MONTH FROM o.created_at) = EXTRACT(MONTH FROM CURRENT_DATE)
                AND EXTRACT(YEAR FROM o.created_at) = EXTRACT(YEAR FROM CURRENT_DATE)
                AND o.state = 0
                AND o.city_id = 13
                AND osl.to_state = 0
                AND osl.channel IN ('1', '4')
            ORDER BY cart_created_at DESC
            ''')

            # Fetch and process data
            records = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            add_to_cart = pd.DataFrame(records, columns=columns)
            logging.info(f"📊 Data retrieved: {len(add_to_cart)} records.")

        # Save Data to CSV
        csv_filename = "add_to_cart.csv"
        add_to_cart.to_csv(csv_filename, index=False)
        logging.info(f"✅ Data saved to {csv_filename}")

        # Upload data to Google Sheets
        def write_df_to_gsheet(gsheet_name, tab_name, df):
            try:
                logging.info("🔄 Authenticating Google Sheets API...")

                if not os.path.exists("summary-automation-project-fd46b6ab2eba.json"):
                    raise FileNotFoundError("❌ Google service account JSON file is missing!")

                gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
                sh = gc.open_by_key("14C3k9An4SSyDAWihuXi_IOC4w8wBLSEa574bAmhDZX0")
                worksheet = sh.worksheet(tab_name)
                set_with_dataframe(worksheet, df)

                logging.info("✅ Google Sheet updated successfully.")
            except Exception as gs_error:
                logging.error(f"❌ Google Sheets Error: {gs_error}")

        write_df_to_gsheet("Add to Cart Customer Details", "Cart_Abondoned_Orders", add_to_cart)

    except Exception as error:
        logging.error(f"❌ Error occurred: {error}")

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()
            logging.info("🔌 PostgreSQL connection closed.")


# Schedule the task to run at a specific time
schedule.every().day.at("13:42").do(cart_abondoned_orders)

logging.info("⏳ Scheduler started. Running every day at 09:30 AM...")
while True:
    try:
        schedule.run_pending()
        time.sleep(5)
    except KeyboardInterrupt:
        logging.info("🛑 Scheduler stopped by user.")
        break
    except Exception as loop_error:
        logging.error(f"❌ Scheduler encountered an error: {loop_error}")


In [ ]:
import schedule
import time
import psycopg2
import pandas as pd
import paramiko
from sshtunnel import SSHTunnelForwarder
import gspread
from gspread_dataframe import set_with_dataframe
import csv
import numpy as np
import re

def cart_abondoned_orders():
    onnection = None  # Initialize connection to None at the start
    cursor = None  # Initialize cursor as None as well
    try:
        # Create an SSH tunnel to the EC2 instance
        with SSHTunnelForwarder(
            ('65.1.183.184', 22),
            ssh_username='ubuntu',
            ssh_pkey="/home/rohit/emami-prod.pem",
            remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
            local_bind_address=('localhost', 5432)
        ) as tunnel:
            # Establish a connection to the RDS PostgreSQL database
            connection = psycopg2.connect(
                host='127.0.0.1',
                port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                database='efrprod',
                user='emamireaduser',
                password='emamireadaccess'
            )
            cursor = connection.cursor()

            # Perform database operations
            cursor.execute('''
            SELECT
                o.id as order_id,
                o.created_at + time '5:30' as cart_created_at,
                --o.user_id,
                o.order_total_paise/100*1.0 as cart_value,
                --o.state,
                --osl.channel,
                u.name,
                ph.number,
                ph.otp_verified_at + time '5:30' as cust_registered_date,
                EXTRACT(DAY FROM o.created_at) as day
            FROM public.orders as o
            left join public.order_status_logs as osl
            on osl.order_id = o.id
            left join users as u
            on u.id = o.user_id
            left join phones as ph
            on ph.user_id = o.user_id
            WHERE
                EXTRACT(MONTH FROM o.created_at) = EXTRACT(MONTH FROM CURRENT_DATE)
                AND EXTRACT(YEAR FROM o.created_at) = EXTRACT(YEAR FROM CURRENT_DATE)
                AND o.state = 0
                AND o.city_id = 13
                AND osl.to_state = 0
                AND osl.channel in ('1','4')
            ORDER BY cart_created_at DESC
            ''')
            records = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]  # Get column names from cursor description
            add_to_cart = pd.DataFrame(records, columns=columns)  # Convert records to DataFrame

        # Save the DataFrame to CSV
        add_to_cart.to_csv('add_to_cart.csv')
        #print("sales_data saved at", pd.Timestamp.now())

        # Update Google Sheets
        gsheet_name = 'Add to Cart Customer Details'  # Google sheet to be updated
        tab_name = 'Cart_Abondoned_Orders'  # Tab to be updated
        df = pd.read_csv("add_to_cart.csv")

        # Function to write data to Google Sheets
        def write_df_to_gsheet(gsheet_name, tab_name, df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("14C3k9An4SSyDAWihuXi_IOC4w8wBLSEa574bAmhDZX0")  # Key of the Google Sheet
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet, df)

        write_df_to_gsheet(gsheet_name, tab_name, df)
        #print("CC_Google_sheet Updated_at", pd.Timestamp.now())

    except (Exception, psycopg2.Error) as error:
        print("Error occurred:", error)

    finally:
        # Ensure the connection is closed if it was successfully opened
        if cursor:
            cursor.close()
        if connection:
            connection.close()
            print("PostgreSQL connection is closed")


# Schedule the task to run every hour at :03
schedule.every().day.at("09:30").do(cart_abondoned_orders)
while True:
    schedule.run_pending()
    time.sleep(5)

# Four Months Data updated at : 5th feb 2025

In [ ]:
#Version : 2.01
#Updates: added promotion details and updation in new sheet (coupon_performance) started 
#Updated_at : 5th March 25 : To change the date
#This Script Fetches last four Month Data - Order date is hard coded need to be update every month 
#else it will keep update more than 4 months order data into Reorder sheet 

# Orders are arranged in ascending order of confirmed on so that older data stays at the top 
# and newer data gets appended below and doesnot disturb CC Agens Disposition/Remarks

import schedule as schedule
import time
import logging

# Configure logging
logging.basicConfig(
    filename="log_four_month_data.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def four_month_data():
    try:
        import csv
        import pandas as pd
        import numpy as np
        import psycopg2
        import pandas as pd
        import numpy as np
        import paramiko
        import re
        from sshtunnel import SSHTunnelForwarder
        import gspread
        from gspread_dataframe import set_with_dataframe
        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                        database='efrprod',
                        user='emamireaduser',
                        password='emamireadaccess'
                )
                cursor = connection.cursor()
        
                # Perform database operations here
                logging.info("Executing DB query to fetch orders data")
                cursor.execute(
                '''
                				SELECT order_Table.*
                ,CASE WHEN
                	od IS NULL THEN 'Not First Order'
                	ELSE '1st Order' END AS new_flag
                ,CASE
                 	WHEN timeout_at IS NOT NULL THEN 'timeout'
                	WHEN rejected_at IS NOT NULL THEN 'rejected'
                	WHEN accepted_at IS NOT NULL THEN 'accepted'
                	ELSE 'pending' end as order_action
                ,promo.coupon_code
                FROM
                (SELECT		
                 osl.order_id		
                ,o.user_id as created_by_id		
                ,CASE		
                	WHEN o.state =0 THEN 'cart'	
                	WHEN o.state =1 THEN 'pre_checkout'	
                	WHEN o.state =2 THEN 'Checkout'	
                	WHEN o.state =6 THEN 'Delivered'	
                	WHEN o.state =7 THEN 'Cancelled'	
                	WHEN o.state =14  THEN 'Return_completed'	
                	ELSE 'In-Progress'	
                END AS status		
                ,o.city_id		
                ,o.delivery_remarks		
                ,o.payment_method		
                ,o.auto_completed		
                ,o.order_total_paise*1.0/100 AS order_value		
                ,o.shipping_total_paise*1.0/100 AS Shipping_charge		
                ,o.doctor_names		
                --,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id		
                ,osl.confirmed_on		
                ,CASE		
                	WHEN osl.channel = 0 THEN 'Call Center'	
                	WHEN osl.channel = 1 THEN 'Mobile'	
                	ELSE 'Website'	
                END as channel_name		
                ,CASE		
                	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'	
                	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                	WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                	WHEN osl.channel = 0 THEN 'Call Center'	
                	WHEN osl.channel = 1 THEN 'Saathi App'	
                	ELSE 'Website'	
                END as channel2		
                ,CASE 		
                	WHEN o.state =0 THEN 'cart'	
                	WHEN o.state =1 THEN 'pre_checkout'	
                	WHEN o.state =2 THEN 'Checkout'	
                	WHEN o.state =3 THEN 'order_received'	
                	WHEN o.state =4 THEN 'shipped'	
                	WHEN o.state =5 THEN 'out_for_delivery'	
                	WHEN o.state =6 THEN 'Delivered'	
                	WHEN o.state =7 THEN 'Cancelled'	
                	WHEN o.state =14 THEN 'Return_completed'	
                	WHEN o.state =17 THEN 'rescheduled'	
                		ELSE 'In-Progress' END AS detailed_status
                ,customer_id		
                ,users.name AS modified_by		
                ,w.code AS fulfillment_center	
                ,Wa.code AS Actual_Mapped_Dc
                ,DS.slot_description		
                ,DS.slot_date AS Expected_Delivery		
                ,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start		
                ,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end		
                ,co.state_changed_on + time '5:30' as cancelled_date		
                ,co.reason		
                ,co.remarks		
                ,CASE		
                	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'	
                	WHEN so.modified_by_id = 25 THEN 'Vinculum API'	
                	WHEN so.modified_by_id = 44306 THEN 'Delite'	
                	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'	
                	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'	
                	ELSE 'Others'END AS shipped_by	
                ,so.state_changed_on + time '5:30'AS shipped		
                ,ofd.state_changed_on + time '5:30'AS out_for_delivery		
                ,del.state_changed_on + time '5:30'AS delivered_date		
                ,iv.amount_paise/100*1.0 AS invoiced_amt		
                ,iv.invoiced_at		
                ,iv.wallet_amount		
                ,ph.number		
                ,us.name AS user_name		
                ,us.created_at AS registration_date		
                ,us.registration_source		
                ,areas.pincode
                ,op.promotion_id
                ,op.promotion_total_paise/100*1.0 as amount_discounted
                ,op.cash_back_total_paise/100*1.0 as cashback
                --,promo.coupon_code
                ,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order		
                --,CASE WHEN 		
                --	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1	
                --	THEN '1st Order' ELSE 'Not First Order'		
                --END AS new_flag_month	
                ,CASE 		
                	 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'		
                	 WHEN del.state_changed_on + time '5:30' 		
                	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'	
                	 WHEN del.state_changed_on + time '5:30' 		
                	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) 	
                	 THEN 'Delay'	
                	 ELSE 'Between Slot'		
                END AS delivery_flag
                ,oah_t.created_at + time '5:30' as timeout_at
                ,oah_a.created_at + time '5:30' as accepted_at
                ,oah_r.created_at + time '5:30' as rejected_at
                
                FROM		
                	(SELECT	
                	order_id	
                	,MAX(modified_by_id) modified_by_id	
                	,MAX(channel) channel	
                	,MAX(state_changed_on + time '5:30') AS confirmed_on	
                	FROM order_status_logs	
                	WHERE state_changed_on >'2024-11-30 18:30' AND to_state = 3	
                	GROUP BY order_id) AS osl	
                LEFT JOIN orders o ON o.id = osl.order_id		
                LEFT JOIN users ON users.id = osl.modified_by_id		
                LEFT JOIN warehouses w ON w.id = o.fulfillment_center_id
                LEFT JOIN warehouses wa ON wa.id = o.store_id
                LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id		
                LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders		
                LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders		
                LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery		
                LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery		
                LEFT JOIN invoices iv ON iv.order_id = osl.order_id		
                LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL		
                LEFT JOIN users us ON us.id = o.user_id 		
                LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id 		
                LEFT JOIN areas ON areas.id = addresses.area_id
                LEFT JOIN order_assignment_histories oah_t on oah_t.order_id = osl.order_id AND oah_t.action = 'time_out'
                LEFT JOIN order_assignment_histories oah_a on oah_a.order_id = osl.order_id AND oah_a.action = 'accept'
                LEFT JOIN order_assignment_histories oah_r on oah_r.order_id = osl.order_id AND oah_r.action = 'reject'
                LEFT JOIN order_promotions op on osl.order_id = op.order_id
                ORDER BY osl.confirmed_on DESC) as order_Table
                LEFT JOIN 
                (SELECT od FROM
                (SELECT ord.*,
                ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number 
                FROM
                (
                SELECT 
                order_id as od
                ,MAX(u.created_at) as registered_at
                ,MAX(o.user_id) as user_id
                ,MAX(state_changed_on + time '5:30') as confirmed_on
                ,MAX(channel) as channel
                ,MAX(modified_by_id) as modified_by
                FROM order_status_logs osl 
                LEFT JOIN orders o on o.id = osl.order_id
                LEFT JOIN users u on u.id = o.user_id
                WHERE to_state = 3 and state_changed_on > '2022-12-31 18:30:00'
                GROUP BY od
                ) AS ord) 
                AS ordr
                WHERE order_number = 1
                AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
                --AND EXTRACT(MONTH FROM confirmed_on) = EXTRACT(MONTH FROM CURRENT_DATE)
                AND confirmed_on >= '2024-10-31 18:30:00'
                --AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE)
                ) as first_order
                on first_order.od = order_Table.order_id
                LEFT JOIN promotions promo on promo.id = order_Table.promotion_id
                order by order_Table.confirmed_on
  
            ''')
                
                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description] # Get column names from cursor description
                df1 = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
                logging.info(f"{len(records)} records fetched and saved to df")
                df = df1[['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
                         'payment_method', 'new_flag', 'order_value', 'shipping_charge',
                         'detailed_status', 'out_for_delivery', 'confirmed_on', 'promotion_id',
                         'shipped', 'fulfillment_center', 'slot_description', 'expected_delivery',
                         'modified_by', 'reason', 'remarks', 'cancelled_date',
                         'delivered_date', 'channel2', 'invoiced_amt', 'shipped_by',
                         'exp_delivery_start', 'exp_delivery_end', 'delivery_flag', 'number','user_name' ,'pincode','wallet_amount'
                         ,'order_action','actual_mapped_dc','coupon_code']]
                logging.info("columns re-arranged in df")

        except (Exception, psycopg2.Error) as error:
            logging.info(f"Error connecting to PostgreSQL : {error}")
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                logging.info("PostgreSQL connection is closed")
        
        logging.info("started updating Reorder sheet")
        gsheet_name = 'Reorder' #This google sheet will be updated
        tab_name = 'C42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1WBLi5ShCGQ1US-x3OxPT2mXmz8sCn5UxiqtXJ16poBU") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f"{gsheet_name} sheet Updated")
        write_df_to_gsheet(gsheet_name,tab_name,df)
        
        gsheet_name = 'coupon_performance' #This google sheet will be updated
        tab_name = 'c42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1E6r9p8Zn5jB-37kQu1Uu_OolOZocrrgOm2yQOtGJtT4") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f"{gsheet_name} sheet Updated")
        write_df_to_gsheet(gsheet_name,tab_name,df)
    
    except (Exception, psycopg2.Error) as error:
        logging.info(f'Error connecting to PostgreSQL:{error}')
    finally:# Close the database connection
        if connection:
            cursor.close()
            connection.close()
            logging.info(f'PostgreSQL connection is closed - job finished\n')

logging.info('Schedule the task to run every hour')
#schedule.every().hours.at(":50").do(four_month_data)
schedule.every(2).hours.at(":30").do(four_month_data)

while True:
    schedule.run_pending()
    time.sleep(5)

In [ ]:
#Version : 2.01
#Updates: added promotion details and updation in new sheet (coupon_performance) started 
#Updated_at : 19th feb 25
#This Script Fetches last four Month Data - Order date is hard coded need to be update every month 
#else it will keep update more than 4 months order data into Reorder sheet 

# Orders are arranged in ascending order of confirmed on so that older data stays at the top 
# and newer data gets appended below and doesnot disturb CC Agens Disposition/Remarks

import schedule as schedule
import time
import logging

# Configure logging
logging.basicConfig(
    filename="log_four_month_data.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def four_month_data():
    try:
        import csv
        import pandas as pd
        import numpy as np
        import psycopg2
        import pandas as pd
        import numpy as np
        import paramiko
        import re
        from sshtunnel import SSHTunnelForwarder
        import gspread
        from gspread_dataframe import set_with_dataframe
        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                        database='efrprod',
                        user='emamireaduser',
                        password='emamireadaccess'
                )
                cursor = connection.cursor()
        
                # Perform database operations here
                logging.info("Executing DB query to fetch orders data")
                cursor.execute(
                '''
                				SELECT order_Table.*
                ,CASE WHEN
                	od IS NULL THEN 'Not First Order'
                	ELSE '1st Order' END AS new_flag
                ,CASE
                 	WHEN timeout_at IS NOT NULL THEN 'timeout'
                	WHEN rejected_at IS NOT NULL THEN 'rejected'
                	WHEN accepted_at IS NOT NULL THEN 'accepted'
                	ELSE 'pending' end as order_action
                ,promo.coupon_code
                FROM
                (SELECT		
                 osl.order_id		
                ,o.user_id as created_by_id		
                ,CASE		
                	WHEN o.state =0 THEN 'cart'	
                	WHEN o.state =1 THEN 'pre_checkout'	
                	WHEN o.state =2 THEN 'Checkout'	
                	WHEN o.state =6 THEN 'Delivered'	
                	WHEN o.state =7 THEN 'Cancelled'	
                	WHEN o.state =14  THEN 'Return_completed'	
                	ELSE 'In-Progress'	
                END AS status		
                ,o.city_id		
                ,o.delivery_remarks		
                ,o.payment_method		
                ,o.auto_completed		
                ,o.order_total_paise*1.0/100 AS order_value		
                ,o.shipping_total_paise*1.0/100 AS Shipping_charge		
                ,o.doctor_names		
                --,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id		
                ,osl.confirmed_on		
                ,CASE		
                	WHEN osl.channel = 0 THEN 'Call Center'	
                	WHEN osl.channel = 1 THEN 'Mobile'	
                	ELSE 'Website'	
                END as channel_name		
                ,CASE		
                	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'	
                	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                	WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                	WHEN osl.channel = 0 THEN 'Call Center'	
                	WHEN osl.channel = 1 THEN 'Saathi App'	
                	ELSE 'Website'	
                END as channel2		
                ,CASE 		
                	WHEN o.state =0 THEN 'cart'	
                	WHEN o.state =1 THEN 'pre_checkout'	
                	WHEN o.state =2 THEN 'Checkout'	
                	WHEN o.state =3 THEN 'order_received'	
                	WHEN o.state =4 THEN 'shipped'	
                	WHEN o.state =5 THEN 'out_for_delivery'	
                	WHEN o.state =6 THEN 'Delivered'	
                	WHEN o.state =7 THEN 'Cancelled'	
                	WHEN o.state =14 THEN 'Return_completed'	
                	WHEN o.state =17 THEN 'rescheduled'	
                		ELSE 'In-Progress' END AS detailed_status
                ,customer_id		
                ,users.name AS modified_by		
                ,w.code AS fulfillment_center	
                ,Wa.code AS Actual_Mapped_Dc
                ,DS.slot_description		
                ,DS.slot_date AS Expected_Delivery		
                ,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start		
                ,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end		
                ,co.state_changed_on + time '5:30' as cancelled_date		
                ,co.reason		
                ,co.remarks		
                ,CASE		
                	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'	
                	WHEN so.modified_by_id = 25 THEN 'Vinculum API'	
                	WHEN so.modified_by_id = 44306 THEN 'Delite'	
                	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'	
                	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'	
                	ELSE 'Others'END AS shipped_by	
                ,so.state_changed_on + time '5:30'AS shipped		
                ,ofd.state_changed_on + time '5:30'AS out_for_delivery		
                ,del.state_changed_on + time '5:30'AS delivered_date		
                ,iv.amount_paise/100*1.0 AS invoiced_amt		
                ,iv.invoiced_at		
                ,iv.wallet_amount		
                ,ph.number		
                ,us.name AS user_name		
                ,us.created_at AS registration_date		
                ,us.registration_source		
                ,areas.pincode
                ,op.promotion_id
                ,op.promotion_total_paise/100*1.0 as amount_discounted
                ,op.cash_back_total_paise/100*1.0 as cashback
                --,promo.coupon_code
                ,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order		
                --,CASE WHEN 		
                --	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1	
                --	THEN '1st Order' ELSE 'Not First Order'		
                --END AS new_flag_month	
                ,CASE 		
                	 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'		
                	 WHEN del.state_changed_on + time '5:30' 		
                	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'	
                	 WHEN del.state_changed_on + time '5:30' 		
                	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) 	
                	 THEN 'Delay'	
                	 ELSE 'Between Slot'		
                END AS delivery_flag
                ,oah_t.created_at + time '5:30' as timeout_at
                ,oah_a.created_at + time '5:30' as accepted_at
                ,oah_r.created_at + time '5:30' as rejected_at
                
                FROM		
                	(SELECT	
                	order_id	
                	,MAX(modified_by_id) modified_by_id	
                	,MAX(channel) channel	
                	,MAX(state_changed_on + time '5:30') AS confirmed_on	
                	FROM order_status_logs	
                	WHERE state_changed_on >'2024-11-30 18:30' AND to_state = 3	
                	GROUP BY order_id) AS osl	
                LEFT JOIN orders o ON o.id = osl.order_id		
                LEFT JOIN users ON users.id = osl.modified_by_id		
                LEFT JOIN warehouses w ON w.id = o.fulfillment_center_id
                LEFT JOIN warehouses wa ON wa.id = o.store_id
                LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id		
                LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders		
                LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders		
                LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery		
                LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery		
                LEFT JOIN invoices iv ON iv.order_id = osl.order_id		
                LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL		
                LEFT JOIN users us ON us.id = o.user_id 		
                LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id 		
                LEFT JOIN areas ON areas.id = addresses.area_id
                LEFT JOIN order_assignment_histories oah_t on oah_t.order_id = osl.order_id AND oah_t.action = 'time_out'
                LEFT JOIN order_assignment_histories oah_a on oah_a.order_id = osl.order_id AND oah_a.action = 'accept'
                LEFT JOIN order_assignment_histories oah_r on oah_r.order_id = osl.order_id AND oah_r.action = 'reject'
                LEFT JOIN order_promotions op on osl.order_id = op.order_id
                ORDER BY osl.confirmed_on DESC) as order_Table
                LEFT JOIN 
                (SELECT od FROM
                (SELECT ord.*,
                ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number 
                FROM
                (
                SELECT 
                order_id as od
                ,MAX(u.created_at) as registered_at
                ,MAX(o.user_id) as user_id
                ,MAX(state_changed_on + time '5:30') as confirmed_on
                ,MAX(channel) as channel
                ,MAX(modified_by_id) as modified_by
                FROM order_status_logs osl 
                LEFT JOIN orders o on o.id = osl.order_id
                LEFT JOIN users u on u.id = o.user_id
                WHERE to_state = 3 and state_changed_on > '2022-12-31 18:30:00'
                GROUP BY od
                ) AS ord) 
                AS ordr
                WHERE order_number = 1
                AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
                --AND EXTRACT(MONTH FROM confirmed_on) = EXTRACT(MONTH FROM CURRENT_DATE)
                AND confirmed_on >= '2024-10-31 18:30:00'
                --AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE)
                ) as first_order
                on first_order.od = order_Table.order_id
                LEFT JOIN promotions promo on promo.id = order_Table.promotion_id
                order by order_Table.confirmed_on
  
            ''')
                
                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description] # Get column names from cursor description
                df1 = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
                logging.info(f"{len(records)} records fetched and saved to df")
                df = df1[['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
                         'payment_method', 'new_flag', 'order_value', 'shipping_charge',
                         'detailed_status', 'out_for_delivery', 'confirmed_on', 'promotion_id',
                         'shipped', 'fulfillment_center', 'slot_description', 'expected_delivery',
                         'modified_by', 'reason', 'remarks', 'cancelled_date',
                         'delivered_date', 'channel2', 'invoiced_amt', 'shipped_by',
                         'exp_delivery_start', 'exp_delivery_end', 'delivery_flag', 'number','user_name' ,'pincode','wallet_amount'
                         ,'order_action','actual_mapped_dc','coupon_code']]
                logging.info("columns re-arranged in df")

        except (Exception, psycopg2.Error) as error:
            logging.info(f"Error connecting to PostgreSQL : {error}")
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                logging.info("PostgreSQL connection is closed")
        
        logging.info("started updating Reorder sheet")
        gsheet_name = 'Reorder' #This google sheet will be updated
        tab_name = 'C42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1WBLi5ShCGQ1US-x3OxPT2mXmz8sCn5UxiqtXJ16poBU") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f"{gsheet_name} sheet Updated")
        write_df_to_gsheet(gsheet_name,tab_name,df)
        
        gsheet_name = 'coupon_performance' #This google sheet will be updated
        tab_name = 'c42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1E6r9p8Zn5jB-37kQu1Uu_OolOZocrrgOm2yQOtGJtT4") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f"{gsheet_name} sheet Updated")
        write_df_to_gsheet(gsheet_name,tab_name,df)
    
    except (Exception, psycopg2.Error) as error:
        logging.info(f'Error connecting to PostgreSQL:{error}')
    finally:# Close the database connection
        if connection:
            cursor.close()
            connection.close()
            logging.info(f'PostgreSQL connection is closed - job finished\n')

logging.info('Schedule the task to run every hour')
#schedule.every().hours.at(":50").do(four_month_data)
schedule.every(2).hours.at(":30").do(four_month_data)

while True:
    schedule.run_pending()
    time.sleep(5)

In [ ]:
#Version : 2
#Updates: added promotion details and updation in new sheet (coupon_performance) started
#Updated_at : 19th feb 25
#This Script Fetches last four Month Data - Order date is hard coded need to be update every month
#else it will keep update more than 4 months order data into Reorder sheet

# Orders are arranged in ascending order of confirmed on so that older data stays at the top
# and newer data gets appended below and doesnot disturb CC Agens Disposition/Remarks

import schedule as schedule
import time
import logging

# Configure logging
logging.basicConfig(
    filename="log_four_month_data.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def four_month_data():
    try:
        import csv
        import pandas as pd
        import numpy as np
        import psycopg2
        import pandas as pd
        import numpy as np
        import paramiko
        import re
        from sshtunnel import SSHTunnelForwarder
        import gspread
        from gspread_dataframe import set_with_dataframe
        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                        database='efrprod',
                        user='emamireaduser',
                        password='emamireadaccess'
                )
                cursor = connection.cursor()

                # Perform database operations here
                logging.info("Executing DB query to fetch orders data")
                cursor.execute(
                '''
                SELECT order_Table.*
                ,CASE WHEN
                	od IS NULL THEN 'Not First Order'
                	ELSE '1st Order' END AS new_flag
                ,CASE
                 	WHEN timeout_at IS NOT NULL THEN 'timeout'
                	WHEN rejected_at IS NOT NULL THEN 'rejected'
                	WHEN accepted_at IS NOT NULL THEN 'accepted'
                	ELSE 'pending' end as order_action
                ,promo.coupon_code
                FROM
                (SELECT
                 osl.order_id
                ,o.user_id as created_by_id
                ,CASE
                	WHEN o.state =0 THEN 'cart'
                	WHEN o.state =1 THEN 'pre_checkout'
                	WHEN o.state =2 THEN 'Checkout'
                	WHEN o.state =6 THEN 'Delivered'
                	WHEN o.state =7 THEN 'Cancelled'
                	WHEN o.state =14  THEN 'Return_completed'
                	ELSE 'In-Progress'
                END AS status
                ,o.city_id
                ,o.delivery_remarks
                ,o.payment_method
                ,o.auto_completed
                ,o.order_total_paise*1.0/100 AS order_value
                ,o.shipping_total_paise*1.0/100 AS Shipping_charge
                ,o.doctor_names
                --,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id
                ,osl.confirmed_on
                ,CASE
                	WHEN osl.channel = 0 THEN 'Call Center'
                	WHEN osl.channel = 1 THEN 'Mobile'
                	ELSE 'Website'
                END as channel_name
                ,CASE
                	WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
                	WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                	WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                	WHEN osl.channel = 0 THEN 'Call Center'
                	WHEN osl.channel = 1 THEN 'Saathi App'
                	ELSE 'Website'
                END as channel2
                ,CASE
                	WHEN o.state =0 THEN 'cart'
                	WHEN o.state =1 THEN 'pre_checkout'
                	WHEN o.state =2 THEN 'Checkout'
                	WHEN o.state =3 THEN 'order_received'
                	WHEN o.state =4 THEN 'shipped'
                	WHEN o.state =5 THEN 'out_for_delivery'
                	WHEN o.state =6 THEN 'Delivered'
                	WHEN o.state =7 THEN 'Cancelled'
                	WHEN o.state =14 THEN 'Return_completed'
                	WHEN o.state =17 THEN 'rescheduled'
                		ELSE 'In-Progress' END AS detailed_status
                ,customer_id
                ,users.name AS modified_by
                ,w.code AS fulfillment_center
                ,Wa.code AS Actual_Mapped_Dc
                ,DS.slot_description
                ,DS.slot_date AS Expected_Delivery
                ,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start
                ,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end
                ,co.state_changed_on + time '5:30' as cancelled_date
                ,co.reason
                ,co.remarks
                ,CASE
                	WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'
                	WHEN so.modified_by_id = 25 THEN 'Vinculum API'
                	WHEN so.modified_by_id = 44306 THEN 'Delite'
                	WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'
                	WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'
                	ELSE 'Others'END AS shipped_by
                ,so.state_changed_on + time '5:30'AS shipped
                ,ofd.state_changed_on + time '5:30'AS out_for_delivery
                ,del.state_changed_on + time '5:30'AS delivered_date
                ,iv.amount_paise/100*1.0 AS invoiced_amt
                ,iv.invoiced_at
                ,iv.wallet_amount
                ,ph.number
                ,us.name AS user_name
                ,us.created_at AS registration_date
                ,us.registration_source
                ,areas.pincode
                ,op.promotion_id
                ,op.promotion_total_paise/100*1.0 as amount_discounted
                ,op.cash_back_total_paise/100*1.0 as cashback
                --,promo.coupon_code
                ,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order
                --,CASE WHEN
                --	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1
                --	THEN '1st Order' ELSE 'Not First Order'
                --END AS new_flag_month
                ,CASE
                	 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'
                	 WHEN del.state_changed_on + time '5:30'
                	 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'
                	 WHEN del.state_changed_on + time '5:30'
                	 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME)
                	 THEN 'Delay'
                	 ELSE 'Between Slot'
                END AS delivery_flag
                ,oah_t.created_at + time '5:30' as timeout_at
                ,oah_a.created_at + time '5:30' as accepted_at
                ,oah_r.created_at + time '5:30' as rejected_at

                FROM
                	(SELECT
                	order_id
                	,MAX(modified_by_id) modified_by_id
                	,MAX(channel) channel
                	,MAX(state_changed_on + time '5:30') AS confirmed_on
                	FROM order_status_logs
                	WHERE state_changed_on >'2024-10-31 18:30' AND to_state = 3
                	GROUP BY order_id) AS osl
                LEFT JOIN orders o ON o.id = osl.order_id
                LEFT JOIN users ON users.id = osl.modified_by_id
                LEFT JOIN warehouses w ON w.id = o.fulfillment_center_id
                LEFT JOIN warehouses wa ON wa.id = o.store_id
                LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id
                LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders
                LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders
                LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery
                LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery
                LEFT JOIN invoices iv ON iv.order_id = osl.order_id
                LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL
                LEFT JOIN users us ON us.id = o.user_id
                LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id
                LEFT JOIN areas ON areas.id = addresses.area_id
                LEFT JOIN order_assignment_histories oah_t on oah_t.order_id = osl.order_id AND oah_t.action = 'time_out'
                LEFT JOIN order_assignment_histories oah_a on oah_a.order_id = osl.order_id AND oah_a.action = 'accept'
                LEFT JOIN order_assignment_histories oah_r on oah_r.order_id = osl.order_id AND oah_r.action = 'reject'
                LEFT JOIN order_promotions op on osl.order_id = op.order_id
                ORDER BY osl.confirmed_on DESC) as order_Table
                LEFT JOIN
                (SELECT od FROM
                (SELECT ord.*,
                ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number
                FROM
                (
                SELECT
                order_id as od
                ,MAX(u.created_at) as registered_at
                ,MAX(o.user_id) as user_id
                ,MAX(state_changed_on + time '5:30') as confirmed_on
                ,MAX(channel) as channel
                ,MAX(modified_by_id) as modified_by
                FROM order_status_logs osl
                LEFT JOIN orders o on o.id = osl.order_id
                LEFT JOIN users u on u.id = o.user_id
                WHERE to_state = 3 and state_changed_on > '2022-12-31 18:30:00'
                GROUP BY od
                ) AS ord)
                AS ordr
                WHERE order_number = 1
                AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
                --AND EXTRACT(MONTH FROM confirmed_on) = EXTRACT(MONTH FROM CURRENT_DATE)
                AND confirmed_on >= '2024-10-31 18:30:00'
                --AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE)
                ) as first_order
                on first_order.od = order_Table.order_id
                LEFT JOIN promotions promo on promo.id = order_Table.promotion_id
                order by order_Table.confirmed_on

            ''')

                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description] # Get column names from cursor description
                df1 = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
                logging.info(f"{len(records)} records fetched and saved to df")
                df = df1[['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
                         'payment_method', 'new_flag', 'order_value', 'shipping_charge',
                         'detailed_status', 'out_for_delivery', 'confirmed_on', 'promotion_id',
                         'shipped', 'fulfillment_center', 'slot_description', 'expected_delivery',
                         'modified_by', 'reason', 'remarks', 'cancelled_date',
                         'delivered_date', 'channel2', 'invoiced_amt', 'shipped_by',
                         'exp_delivery_start', 'exp_delivery_end', 'delivery_flag', 'number','user_name' ,'pincode','wallet_amount'
                         ,'order_action','actual_mapped_dc','coupon_code']]
                logging.info("columns re-arranged in df")

        except (Exception, psycopg2.Error) as error:
            logging.info(f"Error connecting to PostgreSQL : {error}")
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                logging.info("PostgreSQL connection is closed")

        logging.info("started updating Reorder sheet")
        gsheet_name = 'Reorder' #This google sheet will be updated
        tab_name = 'C42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1WBLi5ShCGQ1US-x3OxPT2mXmz8sCn5UxiqtXJ16poBU") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f"{gsheet_name} sheet Updated")
        write_df_to_gsheet(gsheet_name,tab_name,df)

        gsheet_name = 'coupon_performance' #This google sheet will be updated
        tab_name = 'c42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1E6r9p8Zn5jB-37kQu1Uu_OolOZocrrgOm2yQOtGJtT4") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f"{gsheet_name} sheet Updated")
        write_df_to_gsheet(gsheet_name,tab_name,df)

    except (Exception, psycopg2.Error) as error:
        logging.info(f'Error connecting to PostgreSQL:')
    finally:# Close the database connection
        if connection:
            cursor.close()
            connection.close()
            logging.info(f'PostgreSQL connection is closed - job finished\n')

logging.info('Schedule the task to run every hour')
schedule.every().hours.at(":52").do(four_month_data)
while True:
    schedule.run_pending()
    time.sleep(5)

In [ ]:
#Version : 1
#Updates: Working fine till 19th feb'25, updated to add promotions details
#Updated_at : 19th feb 25
#This Script Fetches last four Month Data - Order date is hard coded need to be update every month
#else it will keep update more than 4 months order data into Reorder sheet

# Orders are arranged in ascending order of confirmed on so that older data stays at the top
# and newer data gets appended below and doesnot disturb CC Agens Disposition/Remarks

import schedule as schedule
import time
import logging

# Configure logging
logging.basicConfig(
    filename="log_four_month_data.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def four_month_data():
    try:
        import csv
        import pandas as pd
        import numpy as np
        import psycopg2
        import pandas as pd
        import numpy as np
        import paramiko
        import re
        from sshtunnel import SSHTunnelForwarder
        import gspread
        from gspread_dataframe import set_with_dataframe
        try:
            # Create an SSH tunnel to the EC2 instance
            with SSHTunnelForwarder(
                ('65.1.183.184', 22),
                ssh_username='ubuntu',
                ssh_pkey="/home/rohit/emami-prod.pem",
                remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
                local_bind_address=('localhost', 5432)
            ) as tunnel:
                # Establish a connection to the RDS PostgreSQL database
                connection = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,  # This is the local port bound to the SSH tunnel
                        database='efrprod',
                        user='emamireaduser',
                        password='emamireadaccess'
                )
                cursor = connection.cursor()

                # Perform database operations here
                logging.info("Executing DB query to fetch orders data")
                cursor.execute(
                '''
            SELECT order_Table.*
            ,CASE WHEN
                od IS NULL THEN 'Not First Order'
                ELSE '1st Order' END AS new_flag
            ,CASE
                WHEN timeout_at IS NOT NULL THEN 'timeout'
                WHEN rejected_at IS NOT NULL THEN 'rejected'
                WHEN accepted_at IS NOT NULL THEN 'accepted'
                ELSE 'pending' end as order_action
            FROM
            (SELECT
             osl.order_id
            ,o.user_id as created_by_id
            ,CASE
                WHEN o.state =0 THEN 'cart'
                WHEN o.state =1 THEN 'pre_checkout'
                WHEN o.state =2 THEN 'Checkout'
                WHEN o.state =6 THEN 'Delivered'
                WHEN o.state =7 THEN 'Cancelled'
                WHEN o.state =14  THEN 'Return_completed'
                ELSE 'In-Progress'
            END AS status
            ,o.city_id
            ,o.delivery_remarks
            ,o.payment_method
            ,o.auto_completed
            ,o.order_total_paise*1.0/100 AS order_value
            ,o.shipping_total_paise*1.0/100 AS Shipping_charge
            ,o.doctor_names
            --,o.delivery_slot_id, o.store_id, o.shipping_address_id, o.billing_address_id
            ,osl.confirmed_on
            ,CASE
                WHEN osl.channel = 0 THEN 'Call Center'
                WHEN osl.channel = 1 THEN 'Mobile'
                ELSE 'Website'
            END as channel_name
            ,CASE
                WHEN o.customer_id IS NULL AND osl.channel = 1 THEN 'Mobile'
                WHEN osl.channel = 0 AND COALESCE(o.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
                WHEN osl.channel = 0 AND osl.modified_by_id in(610295,383599,383941,436264) then 'Ecom_CC'
                WHEN osl.channel = 0 THEN 'Call Center'
                WHEN osl.channel = 1 THEN 'Saathi App'
                ELSE 'Website'
            END as channel2
            ,CASE
                WHEN o.state =0 THEN 'cart'
                WHEN o.state =1 THEN 'pre_checkout'
                WHEN o.state =2 THEN 'Checkout'
                WHEN o.state =3 THEN 'order_received'
                WHEN o.state =4 THEN 'shipped'
                WHEN o.state =5 THEN 'out_for_delivery'
                WHEN o.state =6 THEN 'Delivered'
                WHEN o.state =7 THEN 'Cancelled'
                WHEN o.state =14 THEN 'Return_completed'
                WHEN o.state =17 THEN 'rescheduled'
                    ELSE 'In-Progress' END AS detailed_status
            ,customer_id
            ,users.name AS modified_by
            ,w.code AS fulfillment_center
            ,Wa.code AS Actual_Mapped_Dc
            ,DS.slot_description
            ,DS.slot_date AS Expected_Delivery
            ,DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) AS exp_delivery_start
            ,DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME) AS exp_delivery_end
            ,co.state_changed_on + time '5:30' as cancelled_date
            ,co.reason
            ,co.remarks
            ,CASE
                WHEN so.modified_by_id = 94098 THEN 'Ecogreen API'
                WHEN so.modified_by_id = 25 THEN 'Vinculum API'
                WHEN so.modified_by_id = 44306 THEN 'Delite'
                WHEN so.modified_by_id = 162007 THEN 'Kumar Rohit'
                WHEN so.modified_by_id = 175710 THEN 'Arun Kumar'
                ELSE 'Others'END AS shipped_by
            ,so.state_changed_on + time '5:30'AS shipped
            ,ofd.state_changed_on + time '5:30'AS out_for_delivery
            ,del.state_changed_on + time '5:30'AS delivered_date
            ,iv.amount_paise/100*1.0 AS invoiced_amt
            ,iv.invoiced_at
            ,iv.wallet_amount
            ,ph.number
            ,us.name AS user_name
            ,us.created_at AS registration_date
            ,us.registration_source
            ,areas.pincode
            ,ROW_NUMBER() OVER(PARTITION BY o.user_id ORDER BY osl.confirmed_on) AS nth_order
            --,CASE WHEN
            --	ROW_NUMBER() OVER (PARTITION BY o.user_id ORDER BY osl.confirmed_on ASC) = 1
            --	THEN '1st Order' ELSE 'Not First Order'
            --END AS new_flag_month
            ,CASE
                 WHEN del.state_changed_on + time '5:30' IS NULL THEN 'Undelivered'
                 WHEN del.state_changed_on + time '5:30'
                 < DS.slot_date + CAST(SPLIT_PART(DS.slot_description, '-', 1) || ':00' AS TIME) THEN 'Early'
                 WHEN del.state_changed_on + time '5:30'
                 > DS.slot_date + CAST(TRIM(SPLIT_PART(DS.slot_description, '-', 2)) || ':00' AS TIME)
                 THEN 'Delay'
                 ELSE 'Between Slot'
            END AS delivery_flag
            ,oah_t.created_at + time '5:30' as timeout_at
            ,oah_a.created_at + time '5:30' as accepted_at
            ,oah_r.created_at + time '5:30' as rejected_at

            FROM
                (SELECT
                order_id
                ,MAX(modified_by_id) modified_by_id
                ,MAX(channel) channel
                ,MAX(state_changed_on + time '5:30') AS confirmed_on
                FROM order_status_logs
                WHERE state_changed_on >'2024-10-31 18:30' AND to_state = 3
                GROUP BY order_id) AS osl
            LEFT JOIN orders o ON o.id = osl.order_id
            LEFT JOIN users ON users.id = osl.modified_by_id
            LEFT JOIN warehouses w ON w.id = o.fulfillment_center_id
            LEFT JOIN warehouses wa ON wa.id = o.store_id
            LEFT JOIN delivery_slots DS ON o.delivery_slot_id = DS.id
            LEFT JOIN order_status_logs co ON co.order_id = osl.order_id AND co.to_state = 7 -- Calcelled Orders
            LEFT JOIN order_status_logs so ON so.order_id = osl.order_id AND so.to_state = 4 -- Shipped Orders
            LEFT JOIN order_status_logs ofd ON ofd.order_id = osl.order_id AND ofd.to_state = 5 -- out for delivery
            LEFT JOIN order_status_logs del ON del.order_id = osl.order_id AND del.to_state = 6 -- out for delivery
            LEFT JOIN invoices iv ON iv.order_id = osl.order_id
            LEFT JOIN phones ph ON ph.user_id = o.user_id AND ph.deleted_at IS NULL
            LEFT JOIN users us ON us.id = o.user_id
            LEFT JOIN addresses ON addresses.id = o.shipping_address_id --to Get the area id
            LEFT JOIN areas ON areas.id = addresses.area_id
            LEFT JOIN order_assignment_histories oah_t on oah_t.order_id = osl.order_id AND oah_t.action = 'time_out'
            LEFT JOIN order_assignment_histories oah_a on oah_a.order_id = osl.order_id AND oah_a.action = 'accept'
            LEFT JOIN order_assignment_histories oah_r on oah_r.order_id = osl.order_id AND oah_r.action = 'reject'
            ORDER BY osl.confirmed_on DESC) as order_Table
            LEFT JOIN
            (SELECT od FROM
            (SELECT ord.*,
            ROW_NUMBER() OVER (PARTITION BY ord.user_id ORDER BY ord.confirmed_on) AS order_number
            FROM
            (
            SELECT
            order_id as od
            ,MAX(u.created_at) as registered_at
            ,MAX(o.user_id) as user_id
            ,MAX(state_changed_on + time '5:30') as confirmed_on
            ,MAX(channel) as channel
            ,MAX(modified_by_id) as modified_by
            FROM order_status_logs osl
            LEFT JOIN orders o on o.id = osl.order_id
            LEFT JOIN users u on u.id = o.user_id
            WHERE to_state = 3 and state_changed_on > '2022-12-31 18:30:00'
            GROUP BY od
            ) AS ord)
            AS ordr
            WHERE order_number = 1
            AND COALESCE(registered_at, '1970-01-01') >= CURRENT_DATE - INTERVAL '12 MONTH'
            --AND EXTRACT(MONTH FROM confirmed_on) = EXTRACT(MONTH FROM CURRENT_DATE)
            AND confirmed_on >= '2024-10-31 18:30:00'
            --AND EXTRACT(YEAR FROM confirmed_on) = EXTRACT(YEAR FROM CURRENT_DATE)
            ) as first_order
            on first_order.od = order_Table.order_id
            order by order_Table.confirmed_on
                ''')

                records = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description] # Get column names from cursor description
                df1 = pd.DataFrame(records, columns=columns) # Convert records to DataFrame
                logging.info(f"{len(records)} records fetched and saved to df")
                df = df1[['order_id', 'created_by_id', 'status', 'city_id', 'delivery_remarks',
                         'payment_method', 'new_flag', 'order_value', 'shipping_charge',
                         'detailed_status', 'out_for_delivery', 'confirmed_on', 'channel_name',
                         'shipped', 'fulfillment_center', 'slot_description', 'expected_delivery',
                         'modified_by', 'reason', 'remarks', 'cancelled_date',
                         'delivered_date', 'channel2', 'invoiced_amt', 'shipped_by',
                         'exp_delivery_start', 'exp_delivery_end', 'delivery_flag', 'number','user_name' ,'pincode','wallet_amount'
                         ,'order_action','actual_mapped_dc']]
                logging.info("columns re-arranged in df")

        except (Exception, psycopg2.Error) as error:
            logging.info(f"Error connecting to PostgreSQL : {error}")
        finally:
            # Close the database connection
            if connection:
                cursor.close()
                connection.close()
                logging.info("PostgreSQL connection is closed")

        logging.info("started updating Reorder sheet")
        gsheet_name = 'Reorder' #This google sheet will be updated
        tab_name = 'C42' # This particular tab is to be updated
        def write_df_to_gsheet (gsheet_name,tab_name,df):
            gc = gspread.service_account(filename="summary-automation-project-fd46b6ab2eba.json")
            sh = gc.open_by_key("1WBLi5ShCGQ1US-x3OxPT2mXmz8sCn5UxiqtXJ16poBU") #Key Of the google sheet - Summary_Epharmacy
            worksheet = sh.worksheet(tab_name)
            set_with_dataframe(worksheet,df)
            logging.info(f"{gsheet_name} sheet Updated")
        write_df_to_gsheet(gsheet_name,tab_name,df)

    except (Exception, psycopg2.Error) as error:
        logging.info(f'Error connecting to PostgreSQL:')
    finally:# Close the database connection
        if connection:
            cursor.close()
            connection.close()
            logging.info(f'PostgreSQL connection is closed - job finished\n')

logging.info('Schedule the task to run every hour')
schedule.every().hours.at(":02").do(four_month_data)
while True:
    schedule.run_pending()
    time.sleep(5)